## 5. Advanced RAG 기법

> Advanced RAG는 Naive RAG의 단순한 검색 → 생성 구조를 확장하여,  
> 검색 품질을 정교하게 향상시키고, 문맥 전달·정렬·필터링·압축을 최적화하여  
> LLM이 더 정확한 근거 기반 응답을 생성하도록 설계된 RAG 기법들의 집합입니다.

<img src="image/paradigms_of_RAG.png" width="600">

### 5.1 고급 Chunking 기법

RAG에서 **가장 먼저 손봐야 하는 곳**이 바로 Chunking입니다.  
동일한 임베딩 모델·벡터DB·LLM을 쓰더라도, **어떻게 문서를 쪼개느냐에 따라 검색 품질이 극단적으로 달라집니다.**

이 절에서는 다음 세 가지를 다룹니다.

1. 고정 길이 청크 (Fixed-size Chunking)  
2. 재귀적 청크 (Recursive Text Splitting)  
3. 구조 기반 청크 (Structure-aware Chunking)  

#### 5.1.1 왜 Chunking이 중요한가?

임베딩 모델과 벡터DB는 **청크 단위로 검색**을 한다.

- 벡터DB에 저장되는 기본 단위: **청크(chunk)**  
- 검색은 문서 전체가 아니라 청크 벡터들을 대상으로 수행  
- 따라서,
  - 청크가 너무 길면 → 하나의 청크 안에 여러 주제가 섞여 **노이즈 증가**  
  - 청크가 너무 짧으면 → 의미 단위가 깨져서 **문맥 정보 손실**

**좋은 청크의 조건**

1. **완결된 의미 단위**여야 한다. (문단·섹션 단위 등)  
2. **너무 길지도, 너무 짧지도 않아야** 한다. (보통 200~500 tokens 권장)  
3. 서로 **적당히 겹침(overlap)** 이 있어 컨텍스트를 안정적으로 이어줌

이제 각각의 대표적인 청크 방식과 실습 코드를 보겠습니다.

#### 5.1.2 고정 길이 청크 (Fixed-size Chunking)

가장 단순하고 이해하기 쉬운 방식입니다.

- 일정 길이(문자 수 또는 토큰 수) 기준으로 **앞에서부터 잘라 나가는 방식**
- 장점
  - 구현이 매우 쉬움
  - 데이터 형식과 상관없이 동일한 전략 적용 가능
- 단점
  - 문장이 중간에서 끊길 수 있음
  - 한 청크 = 한 의미가 잘 안 맞음
  - 문단/제목/섹션 등 문서 구조를 전혀 고려하지 않음

> **실무에서는 기본값 정도로만 사용하고,  
> 실제 서비스에서는 재귀/구조 기반 전략과 함께 튜닝하는 편이 좋다.**

##### ① 실습: PDF 로드 + 전체 텍스트 합치기

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

PDF_PATH = "data/신입사원민수.pdf"

loader = PyPDFLoader(PDF_PATH)
docs = loader.load()  # 페이지 단위 Document 리스트

print("페이지 수:", len(docs))
print("첫 페이지 내용 샘플:\n", docs[0].page_content[:500])

- `docs`는 `Document` 객체 리스트이며, 각 객체는 `page_content`와 `metadata`를 가집니다.
- 이제 모든 페이지를 하나의 문자열로 합칩니다.

In [ ]:
full_text = "\n".join([doc.page_content for doc in docs])
print("전체 텍스트 길이(문자 수):", len(full_text))

##### ② 실습: CharacterTextSplitter로 고정 길이 청크

`CharacterTextSplitter`는 지정한 기준(separator)으로 먼저 텍스트를 나눈 뒤,  
그 결과로 생성된 조각들을 설정한 chunk_size를 초과하지 않는 범위까지 차례대로 합쳐  
하나의 청크를 생성합니다.

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

fixed_splitter = CharacterTextSplitter(
    separator="\n",    # 줄바꿈을 기준으로 우선 쪼개고, (변경 가능)
    chunk_size=500,    # 최대 8 00자
    chunk_overlap=100,  # 앞 청크의 마지막 50자를 다음 청크와 겹쳐서 포함
    length_function=len
)

fixed_chunks = fixed_splitter.split_text(full_text)

print("고정 길이 청크 개수:", len(fixed_chunks))
print("첫 번째 청크 길이:", len(fixed_chunks[0]))
print("첫 번째 청크 내용 예시:\n", fixed_chunks[0])

- `chunk_size`와 `chunk_overlap`은 나중에 성능 튜닝의 핵심 하이퍼파라미터가 됩니다.
- `separator="\n"` 덕분에 가능한 한 줄 단위로 쪼개고, 그래도 크면 설정한 글자수 기준으로 자릅니다.
- 이 방식은 정확하진 않지만 아무 문서에나 바로 적용 가능하다는 장점이 있습니다.

#### 5.1.3 재귀적 청크 (Recursive Text Splitting)

`RecursiveCharacterTextSplitter`는 **큰 단위 → 작은 단위** 순으로 텍스트를 쪼개면서 `chunk_size`를 넘지 않도록 조절하는 전략입니다.

분할 단위는 다음과 같은 **우선순위(separators)** 로 처리됩니다:

1. **문단 단위**: `"\n\n"`
2. **문장/줄 단위**: `"\n"`
3. **단어 단위**: `" "`
4. **문자(char) 단위**: `""` (더 이상 나눌 수 없을 때 강제 분할)

**동작 원리(간단 버전)**

1. **가장 큰 separator(`\n\n`)로 먼저 텍스트를 분할한다.**  
2. 분리된 조각들 중 **chunk_size(예: 300)를 넘는 조각만** 다음 separator로 다시 분할한다.  
3. 다음 separator(`\n`)로 분할해도 크다면 → 단어 단위(`" "`)로 분할한다.  
4. 그래도 크다면 → 문자 단위로 강제 분할한다.  
5. 모든 조각이 chunk_size 이하가 될 때까지 위 과정을 반복한다.

➡ 결과적으로 텍스트는 **의미 단절을 최소화하면서도 chunk_size에 최대한 근접한 크기**로 분할됩니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=[
        "\n\n",  # 문단 단위
        "\n",    # 줄 단위
        ". ",    # 문장 단위 (간단한 예시)
        " ",     # 단어 단위
        ""       # 문자 단위 (최후의 수단)
    ]
)

recursive_chunks = recursive_splitter.split_text(full_text)

print("재귀적 청크 개수:", len(recursive_chunks))
print("첫 번째 재귀 청크 길이:", len(recursive_chunks[0]))
print("첫 번째 재귀 청크 예시:\n", recursive_chunks[0])

##### ② Fixed vs Recursive 비교 포인트

실습 후 아래와 같이 비교해보겠습니다

In [ ]:
print("고정 청크 개수:", len(fixed_chunks))
print("재귀 청크 개수:", len(recursive_chunks))

print("\n[고정 청크 예시]")
print(fixed_chunks[0])

print("\n[재귀 청크 예시]")
print(recursive_chunks[0])

결과가 똑같아서 당황하셨죠? 코드는 맞습니다. 범인은 **PDF의 강제 줄바꿈**입니다.

우리가 쓰는 PDF 데이터는 문장이 안 끝났는데도 줄이 뚝뚝 끊겨 있는 '하드 랩(Hard Wrap)' 상태입니다.  
그래서 똑똑한 Recursive Splitter도 문맥을 못 보고 그냥 줄바꿈(\n)마다 잘라버린 거죠.

자, 이제 Splitter에게 명령을 바꿔봅시다. '줄바꿈(\n)은 무시하고, 마침표(.)를 찾아서 문장 단위로 뭉쳐라!' 설정을 바꾸고 다시 돌려볼까요?

비교해볼 때 관찰 포인트:

- 문단이 중간에서 끊기는지 여부  
- 한 청크 안에 주제가 여러 개 섞여 있는지  
- 첫 문장/마지막 문장이 자연스럽게 끝나는지  

#### 5.1.4 문서 구조 기반 청크 (Structure-aware Chunking)

보고서·매뉴얼·가이드라인처럼 **문서 구조(Heading, 표제어, 섹션)** 가 중요할 때 매우 효과적.  
제 1장, 제 2장 등 특정 형식으로 되어있을 경우 이 형식을 이용하여 효과적으로 분할 가능

- **방식:**  
  - H1/H2/H3, Section Title 등으로 먼저 분할  
  - 이후 필요한 길이만큼 세부 조정

- **장점:**  
  - 검색 시 특정 "섹션 전체"가 안정적으로 반환  
  - 기업/공공기관 문서에 특히 강력  
  - "근거 제시" 품질이 매우 높아짐

- **적합한 경우:**  
  - 규정집, 공장 매뉴얼, 기술 문서, SOP, 제안서, 법령 등

##### ② 실습: 매우 단순한 헤더 기반 분리 예시

In [ ]:
import re

def split_by_headings(text: str):
    # 정규식 패턴: 줄바꿈 뒤에 "제1장" 또는 "1." 등이 오는 경우 포착
    pattern = r"(?:^|\n)(제\s*\d+장.+|\d+\.\s.+)"
    splits = re.split(pattern, text)
    
    sections = []
    
    # [1] 첫 번째 요소는 '제1장' 나오기 전의 서문(Preamble)입니다.
    # 필요하다면 저장하고, 아니면 건너뜁니다.
    preamble = splits[0].strip()
    if preamble:
        sections.append(f"[서문]\n{preamble}")

    # [2] 1번 인덱스부터 끝까지 2개씩 건너뛰며 (헤더, 본문)을 짝짓습니다.
    # splits[1] = 헤더1, splits[2] = 본문1
    # splits[3] = 헤더2, splits[4] = 본문2 ...
    for i in range(1, len(splits), 2):
        header = splits[i].strip()  # 제1장 ...
        
        # 헤더 뒤에 본문이 있는지 확인 (리스트 범위 체크)
        if i + 1 < len(splits):
            body = splits[i + 1].strip() # 본문 내용
            section_text = f"{header}\n{body}"
            sections.append(section_text)
        else:
            # 마지막에 헤더만 있고 본문이 없는 경우 (거의 없겠지만)
            sections.append(header)

    return sections

In [ ]:
# 테스트 실행
structured_sections = split_by_headings(full_text)

print(f"총 섹션 개수: {len(structured_sections)}")
print("\n=== 첫 번째 섹션 (제1장) 확인 ===")
print(structured_sections[1][:200]) # 0번은 서문일 수 있으므로 1번 확인

print("\n=== 두 번째 섹션 (제2장) 확인 ===")
print(structured_sections[2][:200])

In [ ]:
structured_sections

##### ③ 각 섹션에 재귀 청크 적용

구조 기반으로 나눈 덩어리가 여전히 너무 길다면, 그 안에서 한 번 더 잘게 쪼갭니다.  
마치 반찬(챕터)이 서로 섞이지 않게 '칸막이'를 먼저 치고,   
그 안에서 먹기 좋게 '소분'하는 것과 같습니다.  
이렇게 하면 제1장의 내용과 제2장의 내용이 한 청크에 섞이는 문제를 막을 수 있습니다.  

In [ ]:
structured_chunks = []
for sec in structured_sections:
    chunks = recursive_splitter.split_text(sec)
    structured_chunks.extend(chunks)

print("구조 기반 최종 청크 개수:", len(structured_chunks))
print("구조 기반 첫 청크 예시:\n", structured_chunks[0])

In [ ]:
structured_chunks[:5]

이 방식의 장점:

- 같은 섹션 안에서만 잘린 청크라 문맥이 훨씬 안정적  
- 검색 시, 특정 섹션과 관련된 질의가 왔을 때 **관련 내용이 묶여서 잘 검색**됨  
- 공공문서 / 매뉴얼 / 규정집에서 특히 강력한 효과

#### 5.1.5 정리

이 절에서 한 것:

1. `CharacterTextSplitter`로 **고정 길이 청크** 만들어 보기  
2. `RecursiveCharacterTextSplitter`로 **의미 보존 + 길이 제약을 동시에 고려한 청크** 만들기  
3. 간단한 정규식을 이용해 **문서 구조를 의식한 청크** 만들어 보기  

결국, 어떤 청킹 전략을 선택하여 벡터 DB를 구축했느냐가 검색(Retrieval) 성능의 첫 단추를 끼우는 핵심입니다.  
동일한 문서라도 청킹 방식에 따라 검색 결과의 품질은 달라집니다.

##### 5.1.6 벡터 DB 생성

이제 본격적인 검색(Retrieval) 실습을 위해 벡터 DB를 확정하여 생성합니다.

이 단계에서는 불필요한 재계산을 막기 위해, 로컬에 저장된 DB가 있는지 확인하는 로직을 추가합니다.
- 기존 DB가 있는 경우: FAISS.load_local을 통해 즉시 불러옵니다.
- 기존 DB가 없는 경우: PDF 로드 → 청크 분할 → 임베딩 과정을 거쳐 새로 생성하고 로컬에 저장합니다.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()

# 1. 문서 로드 & 청크 분할
PDF_PATH = "data/신입사원민수.pdf"  
DB_PATH = "./faiss_index_minsu"

# 임베딩 모델 준비
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. 로컬 DB 존재 여부 확인 및 분기 처리
if os.path.exists(DB_PATH):
    # (A) 이미 저장된 DB가 있는 경우 -> 불러오기
    print(f"기존 벡터 DB를 '{DB_PATH}'에서 불러옵니다...")
    
    vectorstore = FAISS.load_local(
        folder_path=DB_PATH, 
        embeddings=embeddings, 
        allow_dangerous_deserialization=True # 필수 설정
    )
    print("DB 로드 완료!")

else:
    # (B) 저장된 DB가 없는 경우 -> 새로 생성 및 저장
    print("저장된 DB가 없습니다. 문서를 로드하고 새로 생성합니다...")

    # --- 문서 로드 & 청크 분할 (기존 코드) ---
    loader = PyPDFLoader(PDF_PATH)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        separators=["\n\n", "\n", " ", ""],
    )

    split_docs = text_splitter.split_documents(docs)
    
    # 메타데이터 ID 부여
    for idx, d in enumerate(split_docs):
        d.metadata["id"] = idx

    print(f"원본 페이지 수: {len(docs)}")
    print(f"분할된 청크 수: {len(split_docs)}")
    
    # --- 벡터 DB 생성 및 저장 ---
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    
    # 로컬에 저장
    vectorstore.save_local(DB_PATH)
    print(f"벡터 DB가 '{DB_PATH}' 경로에 저장되었습니다.")

# Retriever 생성 (RAG에서 사용할 검색기)
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 4}  # 관련도 높은 청크 4개 정도 가져오기
)

### 5.2 Pre-Retrieval 최적화: Query Transformation

지금까지 우리는 **'문서(Document)'를** 잘 쪼개고 저장하는 과정(Indexing)에 집중했습니다.  
이제 검색을 시작해야 하는데, 검색의 품질을 결정짓는 또 다른 중요한 요소가 있습니다.

바로 **사용자의 질문(Query)** 입니다.

RAG 파이프라인에서 벡터 DB에 검색을 요청하기 직전 단계를 **Pre-Retrieval(검색 전처리)** 단계라고 합니다.  
이 단계의 핵심 목표는 **사용자의 모호한 질문을 찰떡같이 바꿔서 검색 확률을 높이는 것**이며,  
이를 위해 **Query Transformation(질의 변환)** 기술을 사용합니다.

#### 5.2.1 Query Rewriting (LLM 기반)

가장 기본적이면서도 효과적인 기법은 **Query Rewriting**(질의 재작성)입니다.  
사용자의 모호하거나 불완전한 질문을 LLM을 이용해 **검색 친화적(Queryable)인 문장**으로 다시 작성하는 방식입니다.

단순히 문장을 다듬는 것을 넘어, **검색 엔진이 선호하는 키워드를 보강**해주는 것이 핵심입니다.

##### 1. 왜 필요한가? (대표적인 실패 사례)

사람의 일상적인 질문과 저장된 공식 문서 사이에는 **표현의 간극**이 존재하기 때문입니다.

**① 단어 불일치 (Vocabulary Mismatch)**
사용자는 구어체나 약어를 쓰지만, 문서는 격식 있는 전문 용어를 사용하는 경우입니다.

- **질문:** "야근 때 **밥값** 얼마 나와?"
- **문서:** "임직원 **연장 근로 시 식대** 지원 규정 및 한도..."

→ 의미는 같지만 단어(밥값 vs 식대)가 달라 벡터 검색의 정확도(Recall)가 떨어집니다.

**② 맥락 부족 (Context Missing)**
대화 흐름에 의존해 질문이 지나치게 축약된 경우입니다.

- **질문:** "그거 기준이 어떻게 돼?"

→ '그거'가 휴가인지, 비용인지 명시되지 않아 검색 엔진은 무엇을 찾아야 할지 모릅니다.

##### 2. Rewriting 적용 전후 비교

LLM에게 **문서를 검색하기 위해 질문을 구체화해달라**는 지시를 내리면 다음과 같이 변환됩니다.

| 구분 | 내용 | 특징 |
| :--- | :--- | :--- |
| **Before** | "회식비 지원돼?" | 구어체 사용, 구체적 항목 누락 |
| **After** | **부서별 회식 및 단합 활동 시 지원 가능한 비용 한도와 청구 절차** | 공식 용어(단합 활동, 청구 절차) 보강 |

**Rewriting의 효과:**
1.  **모호함 제거:** '지원돼?'와 같은 단순 질문을 '비용 한도와 청구 절차'로 구체화
2.  **도메인 용어 주입:** 문서에 포함될 법한 단어('부서별', '규정')를 미리 생성해 매칭 확률 상승

##### Query Rewriting 실습

먼저, 통상적인 RAG 시스템에서 사용하는 방식입니다.  
사용자의 구어체나 비속어를 **검색하기 쉬운 공식 용어**로 변환하는 데 초점을 맞춥니다.  
보통의 사내 규정집이나 매뉴얼을 검색할 때는 이 방식이 효과적입니다.  

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 프롬프트 개선: 역할을 명확히 하고, 변환 목표(검색 최적화)를 구체화
rewrite_prompt = PromptTemplate.from_template("""
당신은 사내 규정 및 업무 매뉴얼 검색을 돕는 AI 어시스턴트입니다.
사용자의 질문을 벡터 데이터베이스 검색에 적합하도록, '명확한 표준어'와 '전문 용어'를 사용하여 다시 작성하세요.

- 구어체나 비속어는 공식 용어로 변경합니다. (예: 밥값 -> 식대, 법카 -> 법인카드)
- 질문의 의도를 파악하여 검색에 필요한 핵심 키워드를 보강합니다.

질문: {query}
검색용 쿼리:
""")

def rewrite_query(query):
    # .content로 문자열만 깔끔하게 반환
    rewritten = llm.invoke(rewrite_prompt.format(query=query)).content
    return rewritten

# 예시 변경: 신입사원 컨텍스트에 맞는 구어체 질문
query = "법카 야근할 때 밥먹는거 얼마까지 됨?"

new_query = rewrite_query(query)

print(f"Original: {query}")
print(f"Rewritten: {new_query}")

대상 문서가 규정집이 아닌 풍자 소설인 경우, 일반적인 표준어 변환 방식은 '식대'와 '고카페인 알약' 사이의  
간극을 메우지 못해 '데이터-질문 불일치' 문제를 일으킵니다.

따라서 검색 성능을 극대화하려면 문서의 도메인 성격(장르, 톤앤매너)에 맞춰, 현실의 의도를 문서 내 고유한  
은유와 상징으로 번역해 주는 맞춤형 프롬프트 전략이 필수적입니다.

결국 효과적인 쿼리 재작성(Query Rewriting)이란 단순히 문장을 다듬는 기술이 아니라, 타겟 문서의 세계관과  
언어에 맞춰 질문을 재해석(Mapping)하는 과정이어야 합니다.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# LLM 설정 (온도는 0으로 설정하여 일관성 유지)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 프롬프트 개선: '소설 검색'에 맞게 역할과 변환 규칙을 재설정
rewrite_prompt = PromptTemplate.from_template("""
당신은 풍자 소설 <오피스 잔혹 동화: 신입사원 민수>의 내용을 검색하려는 사용자를 돕는 AI입니다.
이 소설은 IT 기업의 현실을 '어린 왕자'에 빗대어 은유적으로 표현하고 있습니다.

사용자의 질문을 벡터 DB 검색에 최적화된 형태로 재작성하세요.
1. 질문의 의도(돈, 야근, 상사 등)를 파악합니다.
2. 소설 속에 등장하는 **관련 등장인물**이나 **상징적 키워드**를 쿼리에 반드시 포함시킵니다.

[변환 규칙 예시]
- 돈, 예산, 법인카드 -> '재무팀장(CFO)', '자산', '5억', '숫자'
- 야근, 퇴근 시간 -> '칼퇴', '로딩 바', '11시 40분', '당직 근무자'
- 밥, 식사, 커피 -> '편의점 사장', '고카페인 알약', '탕비실', '믹스 커피'
- 상사, 임원 -> '낙하산 본부장', '꼰대', '왕'

질문: {query}
검색용 쿼리:
""")

def rewrite_query(query):
    # .content로 문자열만 깔끔하게 반환
    rewritten = llm.invoke(rewrite_prompt.format(query=query)).content
    return rewritten

# 테스트: 신입사원 컨텍스트에 맞는 구어체 질문
query = "법카 야근할 때 밥먹는거 얼마까지 됨?"
new_query = rewrite_query(query)

print(f"Original: {query}")
print(f"Rewritten: {new_query}")

In [ ]:
retriever.invoke(query)

In [ ]:
retriever.invoke(new_query)

위 결과에서 볼 수 있듯이, **질문의 표현이 달라지면 검색 결과 역시 달라집니다.**

검색 단계에서 유의미한 청크를 얻지 못했다면,  
부정확한 답변을 생성하기 전에 **질문을 바꿔 다시 검색해보는 것이 중요합니다.**

이때 Query Rewrite는 필수 기능이라기보다는,  
**검색 품질을 보완하고 RAG 성능을 안정화하기 위한 중요한 보조 기능**으로 활용될 수 있습니다.


#### 5.2.3 Query Expansion (Semantic Query Paraphrasing)

Dense Retrieval에서는 질문을 단일 키워드로만 검색할 경우, 문서 내에 존재하는 다양한 표현 방식이나  
구체적인 묘사를 놓쳐 Recall(재현율) 손실이 발생할 수 있습니다.  

특히 우리가 다루는 데이터처럼 특정 상황(Situation)이나 행동(Behavior) 묘사가 많은 문서에서는,   
단순한 직무명(예: CFO)만으로는 관련된 에피소드를 모두 찾기 어렵습니다.  

이를 보완하기 위해 최신 RAG에서는 질문의 의도를 파악하여, 이를 구체적인 상황 묘사나 풀어서 쓴 문장으로  
확장하는 패러프레이징 기법을 사용합니다.

##### 핵심 개념
- Query Expansion: 원 질문을 의미적으로 재구성하여 다양한 관점의 질문 세트를 생성하는 기술

- 전략: 추상적인 질문을 구체적인 행동, 대사, 상황 묘사로 확장하여 Retriever가 문맥(Context)을 더 잘 파악하도록 유도

- 효과: 사용자가 정확한 용어를 모르더라도, 유사한 상황을 묘사한 문장들을 통해 정답 문서를 찾아낼 수 있음

##### 예시

원본 쿼리: 재무팀장(CFO)의 업무 방식과 특징

패러프레이징된 확장 쿼리 예:

기업의 재무 건전성을 관리하고 자금 계획을 수립하는 최고 재무 책임자의 역할
회사의 자산(Asset) 운용 현황을 파악하고 유동성을 관리하는 업무
부서별 예산을 배정하거나 삭감하고, 비용 집행을 통제하는 권한

이처럼 **서로 다른 표현 방식의 질문을 병렬로 검색**한 뒤 결과를 합쳐 사용하면 검색 성능이 크게 향상됩니다.

##### Query Expansion 실습 [CASE A] : 범용적 쿼리 확장 (General Approach)
LLM이 가진 일반적인 상식(General Knowledge)을 기반으로 질문을 확장합니다.  
특징: 위키피디아나 사전적 정의를 기반으로 확장하므로, 일반적인 매뉴얼이나 뉴스 검색에는 효과적입니다.  
하지만 우리 실습 데이터인 풍자 소설에서는 오히려 엉뚱한 키워드를 생성할 위험이 있습니다.  

In [ ]:
paraphrase_prompt = PromptTemplate.from_template("""
당신은 정보 검색 최적화를 위한 쿼리 확장 전문가입니다.
사용자의 질문이 포괄적이거나 추상적일 경우, 이를 구체적인 하위 내용으로 세분화하여 다시 작성하세요.

[확장 가이드]
1. 정의(Definition): 해당 개념의 사전적, 실무적 정의
2. 구체적 행위(Action): 그 주체가 실제로 수행하는 구체적인 행동이나 업무
3. 관련 키워드(Keywords): 해당 주제와 밀접하게 연관된 핵심 단어 (도구, 대상 등)
4. 구성 요소(Components): 질문을 구성하는 하위 세부 항목

질문: {query}

출력 형식:
- 번호 없이 5개의 확장된 문장을 줄바꿈으로 구분하여 출력
""")

def expand_query_paraphrase(query):
    resp = llm.invoke(paraphrase_prompt.format(query=query)).content
    # 응답을 줄 단위 리스트로 변환
    return [line.strip() for line in resp.split("\n") if line.strip()]

# 테스트
expanded_queries = expand_query_paraphrase("재무팀장(CFO)의 업무 방식")
expanded_queries

##### Query Expansion 실습 [CASE B] 도메인 특화 쿼리 확장 (Domain-Specific Approach)
문서의 특수성(세계관, 톤앤매너)을 반영하여 확장을 수행합니다.  
사전적 정의가 아니라 문서 내에서 사용되는 은유, 상징, 고유한 묘사를 중심으로 질문을 재구성합니다.  

In [ ]:
paraphrase_prompt = PromptTemplate.from_template("""
당신은 소설 <오피스 잔혹 동화>의 세계관을 이해하는 AI입니다.
사용자의 질문을 일반 상식이 아닌, **소설 속에 묘사된 구체적인 행동과 은유**로 확장하세요.

[확장 가이드]
- 직무의 사전적 정의 대신, 캐릭터의 **강박적인 행동**이나 **말버릇**을 묘사
- 소설에 등장하는 **상징적 숫자**나 **특정 사물(Object)**을 포함할 것
- (예: 재무팀장 -> 별, 5억, 숫자, 소유, 진지한 사람)

질문: {query}
확장 결과:
""")

def expand_query_paraphrase(query):
    resp = llm.invoke(paraphrase_prompt.format(query=query)).content
    # 응답을 줄 단위 리스트로 변환
    return [line.strip() for line in resp.split("\n") if line.strip()]

# 테스트
expanded_queries = expand_query_paraphrase("재무팀장(CFO)의 업무 방식")
expanded_queries

만약 우리가 검색하려는 문서가 표준화된 매뉴얼이라면 첫 번째 방식(General)만으로도 충분합니다.  
하지만 지금처럼 **고유한 맥락이 있는 데이터(사내 문서, 문학, 상담 로그 등)** 를 다룰 때는,  
LLM이 '상식'을 내려놓고 '문맥'을 따라가도록 프롬프트를 튜닝하는 것이 RAG 성능의 핵심입니다  

**활용**:  
- 확장 키워드를 기반으로 multi-vector 검색  
- 또는 BM25와 함께 hybrid 검색 구성

#### 5.2.4 Multi-Query Retrieval (다중 질문 검색)

**Multi-Query Retrieval**은 앞선 **5.2.3 Query Expansion(패러프레이징)** 을  
**Retrieval 단계에 자동으로 통합한 구조적 기법**입니다.

여기서는 이 전략을 **실제 검색 시스템에서 자동으로 실행하는 구현 단계**를 다룹니다.

즉, Multi-Query Retrieval은 다음 세 단계를 자동화한 패키지 구조다.

1. **LLM이 원 질문을 여러 관점으로 패러프레이징 생성**
2. **각 쿼리를 Retriever에 독립적으로 질의**
3. **모든 결과를 병합(Fusion)하여 상위 k 문서 추출**

이 방식은 dense retriever가 놓친 문서를 다른 질문 표현이 잡아주는 구조이므로  
**Recall이 크게 증가**하고 RAG 응답 품질의 안정성을 높인다.

##### 예시 흐름

원본 질문:
> 우리 회사 재택근무 규정이 어떻게 돼?

LLM이 자동 생성한 확장 쿼리 예: 단순히 문장만 바꾸는 것이 아니라, 질문에 내포된 '서로 다른 의도'를 구체화
- (근태 관점): "재택근무 시 업무 시간 및 출퇴근 체크 방법은?"-> '인사 규정집(HR)' 검색 유도
- (보안 관점): "외부에서 사내망 접속 시 필수 보안 프로그램 및 VPN 설정"-> '정보보호 가이드(Security)' 검색 유도
- (비용 관점): "재택근무 장비 지원금 및 통신비 청구 절차"-> '경비 처리 매뉴얼(Finance)' 검색 유도

각 질문을 검색 → 결과를 모두 합침 →  RRF(Fusion Scoring)로 중복 제거 + 점수 조합 → 최종 k개 문서 반환.


##### 실습

In [ ]:
import logging
from typing import List
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# 앞선 실습에서 생성된 vectorstore와 retriever를 사용합니다
# retriever = vectorstore.as_retriever()

# 1. Multi-Query 생성용 프롬프트 정의 (도메인 특화)
# 일반적인 MultiQueryRetriever는 단순히 "비슷한 질문을 만들어줘"라고만 하지만,
# 우리는 소설의 특수성(은유, 상징)을 반영해 '장면 묘사'와 '구체적 키워드' 유도
multi_query_prompt = PromptTemplate.from_template("""
당신은 소설 '오피스 잔혹 동화'의 데이터베이스에서 정보를 찾는 검색 봇입니다.
사용자의 질문을 바탕으로, 실제 소설 본문에 등장할 법한 **구체적인 문장이나 묘사**를 찾기 위한 검색 쿼리 3개를 생성하세요.

[작성 가이드]
1. 추상적인 질문(의미, 주제, 상징 등)은 피할 것.
2. 대신 **등장인물의 외양 묘사(색깔, 동물 비유), 행동, 대사**를 포함한 질문을 만들 것.
3. 소설 속 고유 명사나 특징적인 키워드(예: 뱀, 명함, 발목 등)를 적극 활용할 것.

원본 질문: {question}

출력 예시:
- 노란 셔츠를 입고 뱀처럼 묘사되는 등장인물은 누구인가?
- 민수의 발목을 스톡옵션 계약서처럼 휘감는 존재
- 로비에서 명함을 건네며 연봉 협상을 하는 캐릭터
""")

# 2. 로깅 및 파싱 함수 정의 (내부 동작 확인용)
def parse_and_log_queries(llm_output: str) -> List[str]:
    """LLM이 생성한 쿼리를 줄 단위로 분리하고, 콘솔에 출력하여 확인합니다."""
    # 줄바꿈으로 분리하고 빈 줄 제거
    queries = [line.strip() for line in llm_output.split("\n") if line.strip()]
    
    print("\n[AI가 생성한 확장 질문들]")
    for i, q in enumerate(queries, 1):
        print(f"{i}. {q}")
    print("-" * 30)
    
    return queries

def get_unique_union(documents: List[List[Document]]) -> List[Document]:
    """
    여러 쿼리로 검색된 문서 리스트들의 중복을 제거하고 합칩니다 (Fusion).
    """
    flattened_docs = [doc for sublist in documents for doc in sublist]
    
    # 중복 제거 (page_content 기준)
    unique_docs = []
    seen_content = set()
    
    for doc in flattened_docs:
        if doc.page_content not in seen_content:
            unique_docs.append(doc)
            seen_content.add(doc.page_content)
            
    print(f"[검색 결과 병합] 총 {len(flattened_docs)}개 문서 검색됨 -> 중복 제거 후 {len(unique_docs)}개 반환")
    return unique_docs

# 3. Chain 구성 (LCEL)

# 1단계: 쿼리 생성 체인
query_generation_chain = (
    multi_query_prompt 
    | llm 
    | StrOutputParser()
    | parse_and_log_queries # 여기서 생성된 쿼리를 출력하고 리스트로 변환
)

# 2단계: 전체 검색 파이프라인
# map()을 사용하여 리스트에 있는 각 쿼리에 대해 retriever를 병렬 수행합니다.
final_retrieval_chain = (
    query_generation_chain
    | retriever.map()       # [Query1, Query2, Query3] -> [[Docs1], [Docs2], [Docs3]]
    | get_unique_union      # [[Docs1], [Docs2], [Docs3]] -> [Unique Docs]
)

# 4. 실행 테스트
query = "헤드헌터는 어떤 존재야?" 
# 소설 내용: 헤드헌터는 '뱀', '발목을 휘감는 존재', '노란 셔츠' 등으로 묘사됨

retrieved_docs = final_retrieval_chain.invoke({"question": query})

print("\n[최종 검색된 문서 내용 일부]")
for i, doc in enumerate(retrieved_docs[:2]): # 상위 2개만 출력
    print(f"문서 {i+1}: {doc.page_content[:100]}...")

지금까지는 쿼리를 변형해 검색 품질을 높이는 방법을 봤습니다.  
이제는 검색기 자체를 설계/조합하는 HyDE와 Hybrid Retrieval을 살펴봅니다

#### 5.2.5 HyDE (Hypothetical Document Embedding)

HyDE는 질문을 바로 검색에 사용하지 않고, LLM을 통해 **가상의 답변(Hypothetical Document)** 을 먼저 생성한 뒤,  
이 답변을 임베딩하여 문서를 검색하는 기법입니다.

이 접근 방식은 **질문(Query)과 문서(Document)는 벡터 공간에서 서로 다른 위치에 존재할 가능성이 높다**는  
문제의식에서 출발합니다.

- 질문: 짧고, 의문문 형태이며, 추상적인 용어를 사용함 (예: "카페에서 일해도 돼?")
- 실제 문서: 길고, 서술형 형태이며, 구체적이고 전문적인 용어를 사용함 (예: "원격 접속 시 보안 터널링 필수 준수")

이러한 표현의 불일치(Mismatch) 때문에 질문 벡터가 정답 문서 벡터 근처에 도달하지 못하는 경우가 발생합니다.  
HyDE는 LLM이 (비록 사실관계가 틀리더라도) 정답 문서와 유사한 '말투'와 '단어'를 가진 가상의 답변을   
만들어내게 함으로써, 이 간극을 메워줍니다.

##### 예시 : 사내 보안 규정 검색
원본 질문 : "카페에서 와이파이 잡아서 일해도 되나요?"
- 기존 검색의 한계: 실제 규정 문서에는 '카페', '와이파이' 같은 단어가 없을 수 있어 검색에 실패할 확률이 높습니다.

LLM이 생성한 가상 답변 (HyDE):
> "외부 장소에서 업무를 수행할 때는 공용 네트워크 보안 수칙에 따라 반드시 VPN을 연결해야 하며, 화면 보안 필름을 부착해야 합니다.  
인가되지 않은 네트워크 접속은 제한됩니다."

실제 정답 문서 (검색 타겟):  
> [제4조 원격 접속 보안] 사외망 접속 시 인가된 가상 사설망(VPN)을 경유해야 하며,  
개방형 무선 네트워크(Open SSID) 사용은 원칙적으로 금지한다.

##### HyDE의 효과와 한계

**효과:**
- 사용자가 정확한 도메인 용어를 몰라도 검색 성능을 개선할 수 있음
- 질문(Query)과 문서(Document) 간 표현 방식 차이를 LLM이 중간에서 보정
- 실제 문서와 유사한 말투·용어를 가진 가상 답변을 통해 벡터 유사도 상승
- 규정, 매뉴얼, 정책 문서처럼 **질문과 문서 스타일이 크게 다른 경우**에 특히 효과적

**주의사항**
- LLM이 해당 도메인에 대한 이해가 부족한 경우
  - 실제 문서와 무관한 키워드를 생성할 수 있음
  - **이 경우 오히려 검색 성능이 저하될 수 있음**
- 회사 고유 용어, 내부 은어, 프로젝트명 등은
  - HyDE 단독 사용보다 사전 용어 사전, 메타데이터, Hybrid Search와 병행하는 것이 안전함

따라서 HyDE는 **자연어 질문 ↔ 공식 문서 간 간극이 큰 환경에서 선택적으로 사용하는 보조 기법**으로 이해하는 것이 적절합니다.

##### HyDE 실습

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 범용 HyDE 프롬프트 (General-Purpose HyDE Prompt)
# 특정 문서(소설 등)를 언급하지 않고, 일반적인 검색 상황을 가정합니다.
hyde_template = """
사용자의 질문에 대해 답변이 될 수 있는 '가상의 문서(Hypothetical Document)'를 작성해주세요.
실제 사실 여부는 중요하지 않으며, 질문에 답변할 때 사용될 법한 **전문적인 단어와 서술형 문장**을 사용하는 것이 핵심입니다.

질문: {query}
가상 답변:
"""

hyde_prompt = PromptTemplate.from_template(hyde_template)

# 2. 질문 설정  
query = "업무 시간을 단축하거나 잠을 쫓는 데 도움이 되는 물건이 있을까?"

# 3. 가상 답변 생성 (Generation)
# LLM은 일반 상식을 동원해 '카페인', '에너지 드링크', '효율', '집중력' 등의 키워드를 생성할 것입니다.
hypo_document = llm.invoke(hyde_prompt.format(query=query)).content

print(f"=== [HyDE] 생성된 가상 답변 (Hallucination) ===\n{hypo_document}\n")

In [ ]:
# 4. 검색 수행 (Retrieval)
# 생성된 가상 답변의 맥락(카페인, 시간 절약 등)을 이용해 실제 문서 검색
results = vectorstore.similarity_search(hypo_document, k=3)

print("=== [Result] 실제 연결된 문서 내용 ===")
for i, r in enumerate(results):
    print(f"\n[문서 {i+1}]")
    print(r.page_content[:300]) 
    # 예상 결과: '편의점 사장'이 '고카페인 알약'을 팔며 '53분을 절약'한다고 말하는 부분

#### HyDE의 함정: 잘못된 환각(Hallucination)이 검색을 망칠 때

방금 실습한 결과에서 HyDE는 그럴듯한 소설을 썼지만, 검색 결과는 우리가 원했던 **고카페인 알약(편의점 사장)** 이 아니었습니다.

**왜 실패했나요?**

1. **세계관의 불일치 (Context Mismatch):**
   - **LLM의 상상:** "야근, 잠? -> 아! 요즘 유행하는 **스마트 스탠드**나 **타이머** 이야기겠구나!" (일반 현대 소설)
   - **실제 데이터:** "야근, 잠? -> **고카페인 알약**을 파는 편의점 사장 이야기" (오피스 잔혹 동화)
   > LLM이 우리 데이터의 **풍자적/판타지적 성격**을 모르고 일반적인 이야기를 지어냈기 때문입니다.

2. **키워드 오염 (Keyword Pollution):**
   - 가상 문서에 등장한 **스마트**라는 단어 때문에 -> 실제 문서의 **스마트폰(인플루언서 마케터)** 이 검색됨.
   - 가상 문서의 **휴식**, **졸음**이라는 단어 때문에 -> **커피(야근의 행진)** 가 검색됨.

**교훈:**
HyDE를 사용할 때는 단순히 "소설처럼 써줘"가 아니라, **그 소설의 고유한 톤앤매너(풍자, 과장, 비유)** 까지  
프롬프트에 담아야 정확도가 올라갑니다.

In [ ]:
from langchain_core.prompts import PromptTemplate

# 수정된 HyDE 프롬프트: '설명문' 대신 '구체적인 이야기/묘사'를 유도
hyde_template = """
사용자의 질문에 대해, 아래 **[세계관]**을 바탕으로 한 짧은 에피소드(가상의 답변)를 창작해 주세요.

[세계관 설정]
1. 장르: **생택쥐페리의 《어린 왕자》를 오마주한 오피스 풍자 소설**.
2. 배경: 삭막한 **한국의 IT 기업**.
3. 문체: 순수하고 철학적이지만, 내용은 냉혹한 직장 생활을 비유적으로 표현.
   (예: '별'은 '부서', '장미'는 '프로젝트', '어른'은 '꼰대 상사'로 은유)

질문: {query}
가상 답변(세계관 기반 창작):
"""

hyde_prompt = PromptTemplate.from_template(hyde_template)

# 질문: 소설 내용을 찾기 위한 질문
query = "업무 시간을 단축하거나 잠을 쫓는 데 도움이 되는 물건이 있을까?"

# 1. 가상 답변 생성
hypo_document = llm.invoke(hyde_prompt.format(query=query)).content

print(f"=== [HyDE] 생성된 가상 답변 (Narrative Style) ===\n{hypo_document}\n")

# 2. 검색 수행
results = vectorstore.similarity_search(hypo_document, k=3)

print("=== [Result] 실제 연결된 문서 내용 ===")
for i, r in enumerate(results):
    print(f"\n[문서 {i+1}]")
    print(r.page_content[:300])

#### 5.2.6 Hybrid Retrieval

**Hybrid Retrieval**은 Advanced RAG의 핵심 기법 중 하나입니다.  

왜냐하면 **단일 검색 방식(dense-only 또는 keyword-only)** 으로는 현실 세계의 문서를 안정적으로 검색하기 어렵기 때문입니다.



| 방식 | 장점 | 단점 |
|------|------|------|
| 벡터 기반 의미 검색 | 표현이 달라도 의미가 같으면 잘 잡아냄 | ~법 제~조와 같은 키워드 법령·표기 검색에 약함 |
| 키워드 기반 매칭 검색| 키워드 검색에 매우 강함. 법령구문·코드·표기 정확도 높음   | 표현이 조금만 달라도 검색 실패|
| Hybrid | 둘 다 잡아서 가장 높은 Recall | 시스템 복잡성 증가 | 

이 둘은 서로 **보완적인 강점**을 가지기 때문에, Hybrid Retrieval은 두 방식의 장점을 결합해  
**Recall(검색 누락 방지)과 Precision(정확성) 모두를 향상**시키는 전략입니다.

##### Hybrid Retrieval의 구성 방식

Hybrid Retrieval은 서로 다른 검색 방식(Dense / Sparse)의 장점을 결합하기 위해  
여러 가지 구성 방식으로 구현할 수 있습니다.

대표적인 방식은 다음과 같습니다.

| 방식 | 설명 | 특징 |
|------|------|------|
| **Parallel Retrieval** | Dense 검색과 BM25 검색을 각각 수행한 뒤 결과를 단순 병합 | 구현이 단순하지만 최종 순위가 불안정할 수 있음 |
| **Weighted Score Fusion** | 두 검색 방식의 점수를 정규화한 뒤 가중 평균으로 결합 | 가중치($\alpha, \beta$)를 수동으로 조정해야 함 |
| **Reciprocal Rank Fusion (RRF)** | 각 검색 결과의 순위를 기반으로 점수를 계산해 결합 | 가중치 튜닝 없이도 안정적인 성능 |
| **EnsembleRetriever** | LangChain에서 제공하는 하이브리드 검색 조합기 | 내부적으로 여러 검색 전략을 조합 |

이 중 **가장 널리 사용되는 방식은 RRF**이며,  
이후 실습에서는 RRF 기반 Hybrid Retrieval을 중심으로 살펴보겠습니다

#### BM25 (Best Matching 25) 란?

BM25는 우리가 흔히 사용하는 **키워드 검색(Keyword Search)** 기법 중  
가장 널리 쓰이고, 실무적으로도 완성도가 높은 방식입니다.

쉽게 말해, **Ctrl + F의 똑똑한 버전**이라고 이해하면 됩니다.

##### Dense 검색 vs Sparse 검색 비교

- **벡터 검색(Dense Retrieval)**  
  - 단어의 의미와 맥락을 중심으로 유사도를 계산  
  - 예: `상인 ≈ 판매자`, `카페 ≈ 커피숍`

- **BM25 검색(Sparse Retrieval)**  
  - 단어 그 자체가 문서에 등장하는지를 중심으로 점수 계산  
  - 예:  
    - `상인`이라는 단어가 포함된 문서 → 점수 상승  
    - 해당 단어가 없는 문서 → 점수 0

##### 작동 원리 (TF-IDF 기반)

BM25는 기본적으로 **TF-IDF 개념을 확장한 방식**입니다.

- **많이 나오면 좋다 (TF, Term Frequency)**  
  - 검색한 단어가 문서에 자주 등장할수록 점수가 높아짐

- **흔한 단어는 덜 중요하다 (IDF, Inverse Document Frequency)**  
  - `그`, `는`, `다`처럼 모든 문서에 자주 등장하는 단어는 가중치를 낮춤  
  - `헤드헌터`, `53분`, `계약서 4조`처럼 희귀하고 구체적인 단어는 가중치를 높게 부여


##### 왜 필요한가?

의미 기반 검색만 사용할 경우,
- 문서의 **전반적인 주제나 분위기**는 잘 맞추지만
- **특정 고유 명사, 숫자, 약어, 코드명**을 정확히 포함한 문서를 놓치는 경우가 발생할 수 있습니다.

BM25는 이런 상황에서 특히 강점을 가집니다.

- 고유 명사 (회사명, 제품명, 인명)
- 정확한 숫자 (날짜, 시간, 조항 번호)
- 내부 용어, 코드명, 약어

따라서 BM25는 **의미 이해가 아니라, 정확한 단어 일치가 중요한 상황**에서  
압도적으로 강력한 검색 방식입니다.


##### Hybrid Retrieval 실습

In [ ]:
from langchain_community.retrievers import BM25Retriever
# 1. 문서 로드 & 청크 분할
PDF_PATH = "data/신입사원민수.pdf"  

# --- 문서 로드 & 청크 분할 (기존 코드) ---
loader = PyPDFLoader(PDF_PATH)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=[
        "\n\n",  # 문단 단위
        ". ",    # 문장 단위 (간단한 예시)
        " ",     # 단어 단위
        ""       # 문자 단위 (최후의 수단)
    ],
)

split_docs = text_splitter.split_documents(docs)

# 메타데이터 ID 부여
for idx, d in enumerate(split_docs):
    d.metadata["id"] = idx
    
# (3) BM25 sparse retriever 구성
bm25_retriever = BM25Retriever.from_documents(split_docs)
dense_retriever = retriever

#### 5.2.6.1 Parallel Hybrid Retrieval (기본 조합)

Dense 검색과 Sparse 검색을 병렬로 수행한 후 결과를 단순 합칩니다.

In [ ]:
query = "편의점 사장이 파는 53분 절약 알약"

dense_docs = dense_retriever.invoke(query)
sparse_docs = bm25_retriever.invoke(query)

print(f"=== [Dense 검색 결과: {len(dense_docs)}개] ===")
for i, doc in enumerate(dense_docs):
    print(f"[{i+1}] {doc.page_content[:50]}...") # 앞부분만 출력

print(f"\n=== [Sparse(BM25) 검색 결과: {len(sparse_docs)}개] ===")
for i, doc in enumerate(sparse_docs):
    print(f"[{i+1}] {doc.page_content[:50]}...")

In [ ]:
# 3. 단순 합치기 (Naive Merge)
# 주의: 이렇게 단순히 합치면, 두 검색기 모두에서 발견된 문서는 중복될 수 있습니다.
combined_docs = dense_docs + sparse_docs

# (선택 사항) 중복 제거 로직 예시
# unique_docs = list({doc.page_content: doc for doc in combined_docs}.values())

print(f"\n=== [Combined 결과: {len(combined_docs)}개] ===")
for i, doc in enumerate(combined_docs):
    print(f"[{i+1}] {doc.page_content[:50]}...")

→ 단순하지만, 중복이 많고 순위가 섞여 있어 중복 제거 및 정렬이 필요함.

#### 5.2.6.2 Reciprocal Rank Fusion (RRF)

**(검색 단계 Fusion)** 서로 다른 검색 전략의 결과를 합쳐 Recall을 높이는 기법

RRF 공식:
```
score = 1 / (k + rank + 1) 
(구현에서 enumerate 사용으로 rank가 0부터 시작하므로 +1 보정)
```
- rank = 각 검색 결과 내에서의 순위(1위면 1, 10위면 10)  
- k = 순위에 대한 완충 상수. 보통 60 고정   

→ 순위가 높을수록 큰 점수  
→ 두 방식에서 모두 높은 순위를 받을수록 최종 점수 증가

LangChain에는 여러 Retriever 결과를 결합하는 도구가 제공되지만,
RRF의 동작 원리를 명확히 이해하기 위해  
여기서는 Reciprocal Rank Fusion을 직접 구현해 보겠습니다.


In [ ]:
def reciprocal_rank_fusion(result_lists, k=60):
    """
    Reciprocal Rank Fusion(RRF)을 직접 구현한 함수

    result_lists : List[List[Document]]
        여러 retriever(Dense, BM25 등)에서 반환된
        검색 결과 리스트들의 리스트
        예: [dense_results, bm25_results]

    k : int (default=60)
        RRF 점수 계산 시 사용하는 완충 상수(smoothing constant)
        - 검색 순위(rank)의 영향력을 완만하게 조절
        - 문서 개수와는 무관함
    """
    
    # 문서별 점수를 누적하기 위한 딕셔너리
    scores = {} # key : 문서 식별자(doc_id),  value : {"doc": Document 객체, "score": 누적 점수}

    # 각 retriever의 검색 결과를 순회
    for results in result_lists:
        for rank, doc in enumerate(results):
            # RRF 공식 적용: 1 / (k + rank), 구현에서는 rank가 0부터 시작하므로 +1 보정
            score = 1.0 / (k + rank + 1)

            # Document ID 기준으로 점수 누적
            doc_id = doc.metadata.get("id", doc.page_content[:50])

            if doc_id not in scores:
                scores[doc_id] = {"doc": doc, "score": 0}
            scores[doc_id]["score"] += score

    # 점수 기준 정렬
    fused = sorted(scores.values(), key=lambda x: x["score"], reverse=True)
    return [x["doc"] for x in fused]

def hybrid_rrf_retrieve(query, retrievers, k=60):
    """
    여러 retriever를 사용해 Hybrid Retrieval을 수행한 뒤
    RRF로 결과를 결합하는 함수

    Parameters
    ----------
    query : str
        사용자 질의

    retrievers : List[BaseRetriever]
        Dense Retriever, BM25 Retriever 등
        invoke(query)를 지원하는 retriever 리스트

    k : int
        RRF 점수 계산용 상수
    """
    
    # 각 retriever에서 독립적으로 검색 수행
    # 결과는 List[List[Document]] 형태
    results = [r.invoke(query) for r in retrievers]
    
    # RRF를 적용해 결과 결합
    fused_results = reciprocal_rank_fusion(results, k=k)
    
    return fused_results

class HybridRRFRetriever:
    def __init__(self, retrievers, k=60):
        self.retrievers = retrievers
        self.k = k

    def invoke(self, query):
        return hybrid_rrf_retrieve(query, self.retrievers, k=self.k)

# 개별 검색 결과 확인 (비교용)
query = "업무 시간을 단축하거나 잠을 쫓는 데 도움이 되는 물건이 있을까?"

dense_results = dense_retriever.invoke(query)
bm25_results = bm25_retriever.invoke(query)

hybrid_rrf = HybridRRFRetriever(
    retrievers=[dense_retriever, bm25_retriever],
    k=60
)


# Hybrid RRF 검색 수행
hybrid_results = hybrid_rrf.invoke(query)

print("RRF Hybrid 결과 수:", len(hybrid_results))

# 상위 3개 결과 출력
for r in hybrid_results[:3]:
    print("---")
    print(r.page_content[:300])

##### RRF의 장점
- 서로 다른 검색 전략의 결과를 **순위 기반으로 균형 있게 결합**하여 검색 편향을 줄임  
- 점수 스케일에 의존하지 않아 단순 가중 평균 방식보다 **훨씬 안정적인 결과**를 제공  
- Dense / Sparse 검색이 혼재된 실제 Enterprise RAG 환경에서 **가장 널리 선호되는 방식**

※ RRF는 이 단계에서 **중복 제거와 순위 정렬까지만 수행**하며,  
최종적으로 **상위 몇 개의 문서를 사용할지는 이후 단계에서 선택**합니다.

#### 5.2.6.3 Hybrid Retrieval 결과 비교 실험

다음 코드를 사용하면 Dense, Sparse, Hybrid(RRF) 결과를 비교할 수 있습니다.

In [ ]:
def preview_results(label, docs):
    print(f"\n=== {label} (Top 2) ===")
    for d in docs[:2]:
        print("---")
        print(d.page_content[:200].replace('\n',''))

# 질문을 바꾸거나 직접 선정하여 테스트 해보세요
query = "이 이야기에서 AI 인턴은 어떤 역할을 하며 어떤 한계를 가지는가?"
#query = "부서 B-612는 어떤 부서인가?"
#query = "민수가 만난 등장인물 중 '헤드헌터'는 어떤 존재인가?"

dense_res = dense_retriever.invoke(query)
sparse_res = bm25_retriever.invoke(query)

hybrid_res = hybrid_rrf.invoke(query)

preview_results("Dense Only", dense_res)
preview_results("Sparse Only", sparse_res)
preview_results("Hybrid (RRF)", hybrid_res)

**관찰 포인트**

- 어떤 방식이 가장 관련성 높은 청크를 포함하는가?  
- Dense-only에서는 누락되지만 Sparse에서는 잡히는 내용이 있는가?  
- Hybrid가 두 방식의 장점을 잘 결합했는가?

#### 정리 

1. **Dense 검색과 Sparse 검색은 서로 보완적인 검색 전략**
2. Hybrid Retrieval(검색 단계 Fusion)로  
   - Recall↑ (빠트리는 문서 줄어듦)  
   - Precision↑ (더 관련성 있는 문서 상위 배치)
3. RRF(검색 단계 Fusion)가 가장 간단하고 안정적인 결합 방식
4. 실무 RAG에서는 대부분 Hybrid Retrieval을 기본값으로 사용

#### 정리 — Retrieval 단계 (Advanced RAG 관점)

1. **Dense 검색과 Sparse 검색은 서로 보완적인 검색 전략**
2. Hybrid Retrieval은 RAG 파이프라인 중  
   **검색(Retrieval) 단계에서** 서로 다른 검색 결과를 결합하는 방식이며,
   - Recall↑ : 정답에 필요한 문서를 빠트리지 않고 후보군에 포함  
   - Precision↑ : 상대적으로 더 관련성 있는 문서를 상위에 배치
3. RRF는 **검색 단계에서 사용할 수 있는 Fusion 기법 중** 가장 단순하면서도 안정적인 결합 방식
4. 실무 수준의 Advanced RAG 시스템에서는 Retrieval 단계의 기본 구성으로 Hybrid Retrieval을 사용하는 경우가 많음

여기까지가 Advanced RAG 파이프라인에서의 **Retrieval(검색) 단계**입니다.  
이 단계의 목적은 사용자의 질문에 대해 **정답에 필요한 정보를 “최대한 빠짐없이” 후보로 끌어오는 것**,  
즉 **Recall을 최대한 확보하는 것**입니다.

하지만 Retrieval 단계에서 수집된 후보 문서 10개, 20개를  
아무 처리 없이 그대로 LLM에게 전달하면 다음과 같은 문제가 발생할 수 있습니다.

- **Lost in the Middle**  
  LLM은 입력 문맥이 길어질수록 중간에 위치한 중요한 정보를 놓치기 쉽습니다.

- **비용 증가**  
  정답과 직접적인 관련이 없는 정보까지 함께 전달되면 불필요한 토큰 비용이 증가합니다.

- **할루시네이션(Hallucination)**  
  관련 없는 정보(Noise)가 섞일수록 LLM이 질문 의도를 오해하거나 근거 없는 답변을 생성할 가능성이 커집니다.

따라서 Advanced RAG에서는 **Retrieval 단계와 Generation 단계를 명확히 분리**하고,  
Retrieval 이후에 검색 결과를 **정제·재정렬하는 별도의 단계**를 둡니다.

이 단계가 바로 **Post-Retrieval**이며, 다음 절 **Post-Retrieval** 에서는  
Cross-Encoder, LLM 기반 재정렬, RRF 등을 활용해 **검색된 후보 중 가장 관련성 높은 문서만 선별하는 방법**을 다룹니다.

### 5.3 Post-Retrieval Strategy: 검색 후 최적화

앞선 장에서는 Advanced RAG 파이프라인에서의 **Retrieval(검색) 단계**를 다루었습니다.

Retrieval 단계의 목표는 사용자의 질문에 대해  
정답에 필요한 문서를 **어떻게든 빠짐없이 후보군으로 끌어오는 것(Recall 확보)** 이었습니다.

이제 다루는 **Post-Retrieval 단계**는 Retriever가 수집해 온 후보 문서들을  
**LLM이 소화하기 좋은 형태로 정제하고 최적화하는 모든 과정**을 의미합니다.

비유하자면,

- Retrieval이 **광산에서 원석을 캐내는 과정**이라면  
- Post-Retrieval은 **원석 중에서 보석을 골라내고 가공하는 과정**입니다.

#### Post-Retrieval 단계의 핵심 목표

Post-Retrieval의 목적은 단순히 문서를 다시 정렬하는 것이 아니라,  
**LLM이 가장 정확한 답변을 생성할 수 있도록 입력 컨텍스트를 최적화하는 것**입니다.

이를 위해 다음과 같은 목표를 가집니다.

- **Precision(정밀도) 향상**  
  → 실제 정답에 가장 가까운 문서를 최상단에 배치

- **Noise(잡음) 제거**  
  → 검색되었지만 실제로는 관련 없는 문서 제거

- **Context 최적화**  
  → LLM의 Context Window(입력 제한)를 효율적으로 사용



#### Post-Retrieval을 위한 대표적인 테크닉

Post-Retrieval 단계에서는 다양한 최적화 기법이 사용됩니다.

| 기법 | 설명 | 비유 |
|----|----|----|
| **Reranking (재순위화)** | 더 강력한 모델로 후보 문서를 다시 채점하여 순서 재배치 | 서류 합격자를 대상으로 심층 면접 |
| **Context Selection** | 중복되거나 점수가 낮은 문서를 과감히 제거 | 커트라인을 정해 합격자만 남김 |
| **Context Compression** | 문서 전체가 아닌 핵심 문장만 요약·발췌 | 책 전체 대신 요약본 읽기 |

이 중에서도 **RAG 성능 향상에 가장 즉각적이고 확실한 효과**를 보이는 기법이  
바로 **Reranking(재순위화)** 입니다.

#### 5.3.1 Reranking (재순위화)

Reranking은 Retrieval 단계에서 수집된 후보 문서들을 대상으로,  
앞선 검색 단계(Bi-Encoder, BM25)보다 **훨씬 더 정밀하지만 느린 모델**을 사용해  
소수의 문서(Top-K)를 다시 평가하고 순위를 재조정하는 기법입니다.

즉,

- Retrieval = “정답에 필요한 문서를 최대한 넓게 모으는 단계 (Recall 중심)”
- Reranking = “그중에서 무엇이 진짜 중요한지 가려내는 단계 (Precision 중심)”

#### 5.3.2 Cross-Encoder 기반 Reranking

Dense/BM25/Hybrid Retrieval은 모두 임베딩을 독립적으로 생성해 벡터 유사도를 비교하는 **bi-encoder 방식**입니다:

- 문서 → 벡터  
- 쿼리 → 벡터  
- 유사도 점수 계산

하지만 bi-encoder는 문서와 질의를 독립적으로 인코딩하기 때문에 
**정교한 관계**까지 잘 반영하지 못하는 경우가 있습니다.

반면 **Cross-Encoder Reranker**는 이렇게 동작합니다:

```
(query, document) → 함께 입력 → 하나의 점수 출력
```

즉, **쿼리와 문서를 동시에 보고 판단**하기 때문에  
관련성 판단이 훨씬 더 정교하고 정확합니다.

대표 모델:
- Cohere Rerank  
- bge-reranker  
- cross-encoder/ms-marco 모델군

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1) Reranker 모델 선택

#model_name = "BAAI/bge-reranker-large" # 고성능
model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2" # 경량 버전 (빠름)

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
reranker = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
reranker.eval()

# 2) Reranking 함수
def cross_encoder_rerank(query, docs, top_k=None):
    """
    query: 문자열
    docs: Document 객체 리스트 (LangChain)
    top_k: rerank 후 가져올 문서 수
    """

    text_pairs = [(query, d.page_content) for d in docs]

    # Tokenize
    encoded = tokenizer(
        text_pairs,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        scores = reranker(**encoded).logits.squeeze(-1)

    scores = scores.cpu().numpy().tolist()

    # (문서, 점수) 묶어서 정렬
    ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)

    if top_k:
        ranked = ranked[:top_k]

    return ranked

# 3) 예시 실행
query = "자연어 처리를 배우는 이유는 무엇인가?"

# dense_retriever or BM25 결과 10개를 가져왔다고 가정
candidate_docs = dense_retriever.invoke(query)[:10]

reranked = cross_encoder_rerank(query, candidate_docs, top_k=5)

print("=== Cross-Encoder Reranking 결과 ===")
for doc, score in reranked:
    print(f"[Score: {score:.4f}] {doc.page_content[:200]}")
    print("---")

#### 5.3.4 LLM-grounded Reranking

최근 Advanced RAG에서는 **LLM이 검색된 후보 문서들을 직접 평가하여 순위를 매기는 방식**도 활용되고 있습니다.

이 방식은 사전 학습된 Reranker 모델 대신, LLM의 추론 능력(reasoning)을 사용해  
“이 문서가 질문에 얼마나 직접적으로 답이 되는가”를 판단합니다.

예:
```
“다음 문서 중 이 질문에 가장 relevant한 순서로 정렬해줘”
```

**장점**
- LLM의 추론 능력을 활용하여  
  단순 유사도 기반보다 **문맥적·논리적 관련성 판단이 더 정교함**
- 특정 도메인에 대해 별도의 Reranker 모델 학습 없이 바로 적용 가능

**단점**
- 문서 수에 비례해 **비용이 빠르게 증가**
- Cross-Encoder 대비 **추론 속도가 느림**
- 출력 형식 제어 및 일관성 확보가 필요함


따라서 LLM-grounded Reranking은 모든 RAG 파이프라인의 기본 구성이라기보다는,

- 후보 문서 수가 매우 적고 (소수 Top-K)
- 응답 품질이 비용과 속도보다 중요한 경우

에 **선택적으로 사용하는 고급 옵션**으로 이해하는 것이 적절합니다.

#### Reranking 실습: 전체 흐름 비교

In [ ]:
query = "AI 인턴에게 권한을 잘못 설정했을 때 어떤 문제가 발생하는가?"

dense_res = dense_retriever.invoke(query)
sparse_res = bm25_retriever.invoke(query)
rerank_res = cross_encoder_rerank(query, dense_res, top_k=5)
rerank_res = list(map(lambda x : x[0], rerank_res)) # 점수를 제거하고 문서만 남김

def show(label, docs, n=3):
    print(f"\n=== {label} ===")
    for d in docs[:n]:
        print(d.page_content[:200])
        print("---")

show("Dense Only", dense_res)
show("Sparse Only", sparse_res)
show("Cross-Encoder Rerank", rerank_res)

#### 정리

1. Retrieval은 후보를 넓게 모으는 과정(Recall 중심)이고, Reranking은 그 후보들의 진짜 중요도를 다시 매기는 과정(Precision 중심)이다.

2. Cross-Encoder 기반 Reranking은 쿼리와 문서를 함께 판단하기 때문에 가장 정교하고 정확하다.

3. LLM-grounded Reranking은 비용과 속도를 감수할 수 있는 상황에서 최고 품질이 필요할 때 사용하는 선택지다.

4. 실무 RAG에서는 **Hybrid Retrieval + Cross-Encoder Reranking** 조합이 가장 안정적인 기본 구성으로 사용된다.

### 5.4 Context Filtering & Compression (Post-Retrieval)

**Context Filtering & Compression은 검색된 문서 중 실제로 필요한 부분만 선별하고, LLM이 처리하기 좋은 형태로 재구성하는 단계입니다.**  
Retrieval과 Reranking을 거친 뒤에도, 검색된 문서에는 다음과 같은 문제가 남아 있을 수 있습니다:

- 관련성이 낮은 청크가 일부 포함됨  
- 비슷한 내용이 중복되어 맥락이 불필요하게 길어짐  
- 중요한 정보가 긴 문서 내부에 묻혀 있음  
- 전체 길이가 LLM의 컨텍스트 창을 초과하거나, 모델의 주의(attention)가 분산됨  

이러한 이유로 Advanced RAG에서는 **검색된 문서를 더 정제하여 LLM이 활용하기 좋은 형태로 만드는 과정**이 필요하며,  
이를 **Context Filtering & Compression**이라고 부릅니다.

#### 5.4.1 왜 Filtering & Compression이 중요한가?

LLM이 검색된 문서를 그대로 처리하기 어렵다는 점에서 출발합니다.

##### ① 컨텍스트 창 제한  
대형 모델은 컨텍스트 창이 넓어진 반면, 일반적으로는 제한이 존재하며 비용도 증가합니다.  
검색 결과를 모두 넣으면 오히려 정보 밀도가 낮아지고 답변 품질이 떨어질 수 있습니다.

##### ② 불필요한 정보는 모델의 판단을 흐릴 수 있음  
중복 청크, 소제목·부록, 잡음(noisy text) 등이 많을수록  
모델의 attention이 분산되며 핵심 내용을 찾아내기 어려워집니다.

##### ③ 중요한 정보가 길이에 묻혀 reasoning이 실패할 수 있음  
검색된 문서가 항상 원하는 답을 직접 제시하는 것은 아니며, LLM은 필요한 부분을 추론해야 합니다.  
이때 문맥이 과도하게 길면 핵심 근거를 인식하지 못해 추론이 실패할 수 있습니다.

Filtering & Compression은 이러한 문제를 줄이고  
**LLM이 실제 답변에 필요한 정보만 명확하게 바라볼 수 있도록 돕는 과정**입니다.

#### 5.4.2 Context Filtering 기법

##### ✔ (1) Relevance Filtering  
검색된 문서들 중에서 “진짜 질문과 관련된 문서만 남기기”

방법:
- 단순 BM25 매칭 기반 필터  
- 임베딩 유사도 기반 cutoff  
- LLM에게 직접:  
  ```
  쿼리와 관련 없는 문서를 골라 제거하라
  ```

##### ✔ (2) Deduplication (중복 제거)  

같은 내용을 가진 문서가 여러 개 있을 경우 제거.

- 텍스트 유사도 기반  
- 임베딩 유사도 기반  
- Cosine similarity threshold 사용

##### ✔ (3) Segment-level Filtering  
문서 전체가 아니라 문서의 일부만 필요할 경우:

- 슬라이딩 윈도우 기반 필터  
- 문장 단위 임베딩 후 중요 문장만 선택  
- 문서 내부에서 “답변에 중요한 부분만 추출”

#### 5.4.3 Context Compression 기법

##### ✔ (1) Extractive Compression  
문서 중 **핵심 문장만 추출**  
- 문장 임베딩 기반 scoring  
- 범주형 분류 기반 extraction  
- LLM 기반 extractive summarization

##### ✔ (2) Abstractive Compression  
LLM이 문서를 이해한 뒤 **요약된 새로운 문장을 생성**

- "문서 내용을 절반 이하 길이로 요약하라"  
- “이 질문에 필요한 핵심 정보만 남겨라”  

##### ✔ (3) LLM-grounded Compression  
LLM이 문서를 읽고 직접 relevance scoring 을 수행:

질문 + 문서 →  
**이 문서가 필요한지 판단 → 필요하다면 중요한 부분만 압축**

→ 최근 가장 강력한 방식

#### 실습: Contextual Compression Retriever

기본 Retriever는 단순히 유사도가 높은 청크를 여러 개 가져오기만 합니다.  
하지만 이 결과에는 중복된 내용, 불필요한 배경 설명, 질문과 상관없는 정보가 섞여 있을 때가 많습니다.   

이를 해결하기 위해 RAG에서는 검색 후(Post-retrieval) 단계에서 문서를 정제합니다.  
그 대표적인 방식이 Contextual Compression Retriever 입니다.  

이를 활용하여 간단한 Context Filtering & Compression 실습을 진행 하겠습니다.  

#### 실습: Post-Retrieval 문맥 압축 적용하기

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

# LLM 설정: 문서를 "압축 요약"하는 역할만 담당
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 문서 압축용 프롬프트
# - 질문과 관련된 정보만 남기고 불필요한 서술은 제거하도록 안내
compress_prompt = ChatPromptTemplate.from_template("""
당신은 '문서 요약 전문가'입니다.
주어진 [문서]에서 사용자의 [질문]에 답변하는 데 필요한 **핵심 정보**만 추출하여 요약하세요.

질문: {query}

문서:
{content}

출력 규칙:
- 질문과 직접 관련된 내용만 3~5줄로 요약
- 배경 설명, 수식어, 문서 전체 요약은 제외
- 만약 문서에 질문과 관련된 내용이 전혀 없다면 "관련 정보 없음"이라고만 출력
""")

compress_chain = compress_prompt | llm


def compressed_retrieve(dense_retriever, query: str, k: int = 4):
    """
    1) 기본 검색 수행
    2) 검색된 문서를 하나씩 LLM에게 전달해 질문 기준 요약
    3) 요약된 내용으로 Document를 교체하여 반환
    """
    
    docs = dense_retriever.invoke(query)

    compressed_docs = []
    print(f"Original Docs: {len(docs)}개 발견. 압축 시작...")

    for i, d in enumerate(docs):
        response = compress_chain.invoke({
            "query": query,
            "content": d.page_content
        })
        summary = response.content.strip()

        if "관련 정보 없음" not in summary:
            new_doc = Document(
                page_content=summary,
                metadata=d.metadata
            )
            compressed_docs.append(new_doc)
            print(
                f" -> 문서[{i+1}] 압축 완료 "
                f"({len(d.page_content)}자 → {len(summary)}자)"
            )
        else:
            print(f" -> 문서[{i+1}] 관련 내용 없어 필터링됨")

    return compressed_docs[:k]

# 실행 예시
query = "AI 인턴에게 권한을 잘못 설정했을 때 어떤 문제가 발생하는가?"
compressed_docs = compressed_retrieve(dense_retriever, query, k=4)

print("압축된 문서 개수:", len(compressed_docs))
for d in compressed_docs:
    print("---")
    print(d.page_content[:300])


**관찰 포인트**
- 검색된 문서보다 압축된 문서의 길이가 크게 줄어 있는가?  
- 질문과 직접 관련 없는 문서가 제거되었는가?  
- 불필요한 서론/부록/표 등이 사라졌는가?

#### 실습 : 검색 결과 중복 제거 (Post-Retrieval Deduplication)

Retriever가 반환한 문서들은 서로 다른 출처에서 왔더라도,
의미적으로 거의 동일한 내용을 담고 있는 경우가 많습니다.

예:
- "민수는 커피를 마셨다."
- "주인공은 카페인을 섭취했다."

이러한 중복 문서들은
- LLM의 토큰을 불필요하게 소모하고
- 같은 내용을 반복하게 만들어
- 최종 답변의 품질을 떨어뜨릴 수 있습니다.

따라서 Post-Retrieval 단계에서는
이미 검색된 Document 객체들을 대상으로
의미적 중복을 제거하는 Deduplication 과정이 필요합니다.

In [ ]:
import numpy as np
from langchain_openai import OpenAIEmbeddings

# 1. 임베딩 모델 준비 (유사도 계산용)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

def dedupe_after_retrieval(docs, threshold=0.90):
    """
    검색된 문서들(docs) 간의 유사도를 계산하여 중복을 제거합니다.
    - docs: 검색된 Document 객체 리스트
    - threshold: 이 값 이상으로 유사하면 중복으로 간주하고 제거
    """
    if not docs:
        return []

    # (1) 문서 텍스트를 벡터로 변환
    texts = [d.page_content for d in docs]
    vectors = np.array(embeddings.embed_documents(texts))

    # (2) 벡터 정규화 (Cosine Similarity 계산을 위해)
    # 정규화된 벡터끼리의 내적(dot product)은 코사인 유사도와 같습니다.
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    vectors = vectors / (norms + 1e-12) # 0 나누기 방지

    keep_indices = []
    removed_indices = set()

    for i in range(len(docs)):
        if i in removed_indices:
            continue
        
        keep_indices.append(i)

        # 현재 문서(i)와 그 뒤에 있는 모든 문서(i+1 ~ 끝) 비교
        sims = np.dot(vectors[i], vectors[i+1:].T)

        # 유사도가 threshold 넘는 문서들은 '제거 목록'에 추가
        for idx, sim in enumerate(sims):
            if sim > threshold:
                removed_indices.add(i + 1 + idx) # 인덱스 보정

    return [docs[i] for i in keep_indices]


# 테스트: retriever로 문서 검색 → 중복 제거
query = "편의점 사장이 파는 물건의 효능은 무엇인가?"

# 1. (중복이 포함될 수 있는) 넉넉한 개수로 검색
retrieved_docs = dense_retriever.invoke(query)
print("검색된 문서 수:", len(retrieved_docs))

# 2. 중복 제거 실행
deduped_docs = dedupe_after_retrieval(retrieved_docs, threshold=0.9)
print("중복 제거 후 문서 수:", len(deduped_docs))

print("\n[최종 남은 문서 리스트]")
for i, d in enumerate(deduped_docs):
    print(f"[{i+1}] {d.page_content[:100]}...")


#### 정리

1. Retrieval 단계에서 검색된 문서를 그대로 LLM에 전달하는 것은 항상 최적의 선택은 아님.  
2. 불필요한 문서, 중복 정보, 과도한 길이는 답변 품질과 효율을 모두 저하시킬 수 있음.  

3. **Filtering**  
   - 질문과 관련성이 낮은 문서를 제거하거나  
   - 사용 목적에 맞지 않는 청크를 제외하는 과정  
   - 예: reranking 점수 기준, 유사도 기준 컷오프 등  

4. **Compression**  
   - 문서를 제거하지 않되  
   - LLM이 활용하기 좋은 형태로 정보를 재구성  
   - 요약, 핵심 문장 추출, 질문 중심 재서술 등을 포함  

5. **Deduplication**  
   - 내용적으로 거의 동일한 문서가 여러 개 포함되는 경우  
   - 대표 문서 하나만 남기고 중복을 제거하여 입력 문맥을 간결하게 유지  

6. Filtering, Compression, Deduplication을 거친 문맥은  
   - 정보 밀도가 높고  
   - LLM의 추론 부담이 적으며  
   - 제한된 컨텍스트 창을 효율적으로 사용하는 입력이 됨  

다음 절 **5.5 Generator 단계 최적화**에서는,  
이렇게 정제된 문맥을 바탕으로 LLM이 더 정확하고 일관된 답변을 생성하도록  
**프롬프트 구조 설계, 출처(citation) 처리, 출력 제약 조건(output constraints)** 등을 다룹니다.


### 5.5 Generation 단계 최적화

Retrieval / Reranking / Filtering / Compression을 거친 후  
마지막 단계는 **LLM이 실제로 답변을 생성하는 Generation 단계**입니다.

많은 개발자가 “검색 성능이 좋아지면 답변도 좋아진다”고 생각하지만,  
**Generation 단계 자체가 최적화되지 않으면 여전히 다음 문제가 발생할 수 있습니다.**

- 검색된 문서를 제대로 활용하지 못함  
- 답변이 검색 문맥과 동떨어짐 (hallucination)  
- 불필요하게 긴 답변  
- citation(출처 근거) 없음  
- JSON 등 구조화된 출력 실패  

따라서 Advanced RAG에서는 **검색된 컨텍스트를 LLM이 가장 잘 소비하도록 Prompt 구조를 최적화하는 과정**이 반드시 필요하다.

#### 5.5.1 Generation 단계가 어려운 이유

##### ✔ 1) LLM은 문서를 “그대로 반영”하지 않는다  
 - LLM은 문서의 구조적 의미(조건·예외·표 등)를 정확히 파악하는 데 한계가 있어 내용의 핵심을 놓치기 쉽다.  
 - 특히 문서가 길어질수록 attention이 희석되어 일부 정보만 단편적으로 반영된다.  
 - 이로 인해 실제 문서의 기준이나 요건을 잘못 이해하거나 누락하는 오류가 발생한다.

##### ✔ 2) 컨텍스트와 질문 사이의 연결을 모델이 자동으로 맞추지 못함  
 - LLM은 질문에 맞는 답변 패턴을 생성하려는 성향이 강해 문서 내용과 질문의 alignment가 쉽게 어긋난다.  
 - 문서에 없는 내용을 일반 지식을 기반으로 보완하거나, 질문과 무관한 문서의 부분을 가져오는 “semantic drift”가 발생한다.  
 - 프롬프트로 제약을 명확히 주지 않으면 문서 기반 답변이 아니라 추정 기반 답변을 만들어내기 쉽다.

##### ✔ 3) Hallucination은 Generation 단계에서 주로 발생  
 - 문서가 부족하거나 질문이 모호하면 LLM은 빈 부분을 상식이나 추론으로 채우려 하면서 새로운 내용을 만들어낸다.  
 - 문서 길이가 길거나 모델 규모가 작을수록 문맥 유지가 어려워져 오답 생성 확률이 높아진다.  
 - 이 때문에 검색이 정확해도 생성 단계에서 잘못된 정보로 답변이 왜곡될 수 있다.

##### ✔ 4) 개발자가 원하는 출력 형태(JSON, bullet point 등)를 LLM이 항상 지키기 어렵다  
- LLM은 정해진 포맷을 따르는 것보다 자연스러운 문장을 생성하는 방향으로 확률이 치우쳐 있다.  
- 출력 구조가 복잡해질수록 형식을 깨거나 일부 필드를 누락하는 문제가 발생한다.  
- 이는 RAG에서 API 응답 형태나 strict format이 필요한 경우 큰 장애로 이어질 수 있다.

#### 5.5.2 Generation 최적화의 핵심 전략

Advanced RAG에서 Generator 최적화는 크게 4가지로 나뉜다.

- > ① Prompt 구조 최적화  
- > ② Citation / Attribution 기반 생성  
- > ③ Output Constraint (형식 강제)  
- > ④ Fail-safe 답변 전략 (안전성 향상)

#### 5.5.3 Prompt 구조 최적화

좋은 RAG Prompt의 구조는 아래 형태를 따른다:

```
[역할 정의 Role]
당신은 문서 기반 질문응답 시스템입니다.
문서에 기반하지 않은 내용은 절대 답변하지 마십시오.

[질문 Question]
{query}

[컨텍스트 Context]
{context}

[답변 지침 Instructions]
- 문서에 있는 내용만 답변할 것
- 문서에 없는 정보는 "문서에 정보가 없습니다"라고 말할 것
- 핵심 요약 후 필요한 경우 상세 설명 제공
```

#### 실습: 프롬프트 템플릿 구성

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = PromptTemplate.from_template("""
당신은 문서 기반 QA 시스템입니다.
아래 문맥(Context)에 기반해서만 답변하세요.
문맥에 없는 내용은 절대 생성하지 마십시오.

[질문]
{query}

[문맥]
{context}

[출력 형식]
- 핵심 결론을 먼저 말한다.
- 문서에 근거한 사실만 말한다.
- 필요한 경우 문서의 일부 문장을 인용한다.

답변:
""")

#### 5.5.4 Citation / Source Attribution

사용자 신뢰도를 높이기 위해  
**문서의 어느 부분에서 답을 가져왔는지** 표시하도록 할 수도 있습니다.

예시:

```
지원 기준은 다음과 같습니다:
- 기술 검토 절차 필수 (출처: 문서 2번 청크)
- 관련 기관 승인 필요 (출처: 문서 4번 청크)
```

#### 실습: Citation 포함 답변 생성

In [ ]:
def format_context_with_ids(docs):
    combined = ""
    for i, d in enumerate(docs):
        combined += f"[문서{i}]\n{d.page_content}\n\n"
    return combined

query = "편의점 사장이 파는 물건은 무엇이며 어떤 효과가 있는가?"

docs = dense_retriever.invoke(query)
formatted_context = format_context_with_ids(docs)

response = llm.invoke(prompt.format(query=query, context=formatted_context))
print(response.content)

#### 5.5.5 Output Constraint (JSON 구조 강제)

LLM이 정확한 구조화 출력을 해야 하는 경우:

- API 응답 구조 맞추기
- 시스템 간 연동
- 대시보드용 structured data  

등에서 필수적입니다.

#### 예: JSON으로 답변 강제

In [ ]:
json_prompt = PromptTemplate.from_template("""
아래 문맥에 기반하여 질문에 답하십시오.
출력은 반드시 JSON으로만 생성하십시오.
기타 설명 금지.

문맥:
{context}

질문:
{query}

JSON 형식:
{{
    "summary": "...",
    "details": ["...", "..."],
    "confidence": "high | medium | low"
}}
""")

실습:

In [ ]:
resp = llm.invoke(json_prompt.format(query=query, context=formatted_context))
print(resp.content)

#### 5.5.6 Fail-safe 답변 전략

문서에 정보가 없을 때는 다음을 강제해야 한다:

```
문서에 정보가 없습니다. (hallucination 방지)
```

#### 참고: Fail-safe 포함 프롬프트

In [ ]:
safeguard_prompt = PromptTemplate.from_template("""
문서에 근거하지 않은 정보는 만들어내지 마십시오.
문서에 답이 없으면 다음과 같이 말하십시오:
"문서에 해당 정보가 없습니다."

문서:
{context}

질문:
{query}
""")

#### 5.5.7 전체 Generation 흐름 실습

In [ ]:
query = "편의점 사장이 파는 물건은 무엇이며 어떤 효과가 있는가?"

docs = dense_retriever.invoke(query)
context = format_context_with_ids(docs)

response = llm.invoke(
    prompt.format(query=query, context=context)
)

print("=== 최종 답변 ===")
print(response.content)

관찰 포인트:

- 문서에 기반한 답변인가?  
- 문서에 없는 내용을 추가로 말하지 않는가?  
- 핵심 내용을 먼저 말하는가?  

#### 5.5.8 정리

1. 검색 품질이 좋아도 Generation 단계가 최적화되지 않으면 RAG 전체 품질이 떨어짐  
2. Prompt 구조 최적화는 hallucination 방지의 핵심  
3. Citation/Output constraints는 실무에서 필수  
4. Fail-safe 전략으로 안전성 강화  
5. Retrieval 품질 + Generation 최적화가 함께 적용될 때 RAG 성능이 극적으로 향상  

다음 절 에서는 실무에서 매우 중요한 **Retrieval 품질 평가**, **Generation 품질 평가**,  
그리고 Recall@k, nDCG 등 검증 지표와 평가 실습을 다룹니다.

## 6. RAG 성능 평가 (Evaluation)  

**평가(Evaluation)는 RAG에서 가장 중요하면서도, 실제 구현 과정에서는 종종 간과되는 핵심 단계입니다.**  

검색 품질이 충분한지, 생성이 문서 기반으로 이루어지는지, 그리고 전체 파이프라인이 의도한 대로 작동하는지를  

**정량적·정성적 기준으로 체계적으로 검증하는 과정이 반드시 필요합니다.**

<img src="image/eval.png" width="380">

RAG 평가는 크게 두 영역으로 나눌 수 있습니다.

1. **Retrieval Evaluation (검색 품질 평가)**  
2. **Generation Evaluation (생성 품질 평가)**  

이 절에서는 두 평가 방법의 이론적 배경과 실제 적용 절차를 함께 다룹니다.

### 6.1 왜 RAG 평가가 중요한가?

다음과 같은 상황은 모두 **평가가 부실할 때** 흔히 발생하는 문제들입니다.

- 벡터 유사도 기반 검색만 사용하다 보니, 실제로 중요한 문서가 반복적으로 검색되지 않음  
- 검색은 잘 되었는데 LLM이 문서를 근거로 답하지 않고 자체 추론으로 대답함  
- Chunk 크기나 슬라이딩 윈도우 전략을 바꿔도 어떤 설정이 더 효과적인지 판단할 근거가 없음  
- Query Rewriting/Expansion을 적용했지만, 오히려 Recall이 떨어져 성능이 악화됨  
- BM25·Vector·Hybrid Retrieval 실험을 해도 정량적 지표 없이 “감”으로 방식이 선택됨  

따라서 **RAG 파이프라인을 안정적으로 개선하기 위해서는 체계적 평가가 필수적입니다.**

### 6.2 Retrieval Evaluation (검색 품질 평가)

#### 핵심 지표

| 지표 | 설명 |
|------|------|
| **Recall@K** | 정답 문서가 상위 K개 검색 결과 안에 포함되는 비율 |
| **Precision@K** | 상위 K개 검색 결과 중 실제로 정답에 해당되는 문서 비율 |
| **MRR (Mean Reciprocal Rank)** | 정답 문서가 몇 번째에 등장했는지의 역수 평균 |
| **nDCG (Normalized Discounted Cumulative Gain)** | 순위의 품질을 점수 기반으로 평가하는 지표 |

RAG에서 가장 중요한 지표는 **Recall@K** 입니다.  
정답 문서가 검색 단계에서 확보되지 않으면, LLM이 아무리 뛰어나더라도 올바른 답변을 생성하기 어렵습니다.

#### 6.2.1 Retrieval 평가를 위한 데이터 구조

Retrieval 평가는 다음처럼 구성된 데이터셋을 필요로 한다.

```python
test_cases = [
    {
        "query": "농업 기술 보급 기준은?",
        "answer_doc_ids": [3, 7]   # 정답이 들어있는 문서(청크)의 ID
    },
    ...
]
```

각 질문마다 정답이 포함된 청크 ID를 지정해 주면 됩니다.

#### 실습: Recall@K 계산하기

##### ① 검색 함수 준비

In [ ]:
def retrieve_ids(retriever, query, k=5):
    docs = retriever.invoke(query)
    return [d.metadata.get("id") for d in docs[:k]]

※ 전제: 청크 생성 과정에서 metadata에 "id": index 형태로 저장되어 있다고 가정합니다.

In [ ]:
[(doc.metadata['id'], doc.page_content) for doc in split_docs]

In [ ]:
# RAG 성능 평가를 위한 Ground Truth 데이터셋 (정확 매핑만 + 20개 확장)

test_cases = [
    # 1) [구체적 수치] 편의점 사장과 시간 절약
    {
        "query": "편의점 사장이 파는 알약을 먹으면 일주일에 시간을 얼마나 절약할 수 있나?",
        "answer_doc_ids": [73, 74],
    },

    # 2) [반복 요청] 민수가 팀장에게 끈질기게 부탁한 것
    {
        "query": "신입 사원 민수가 팀장에게 반복해서 만들어 달라고 부탁한 것은 무엇인가?",
        "answer_doc_ids": [4, 5, 6, 7],
    },

    # 3) [고유명사] 민수가 근무하던 부서 코드명
    {
        "query": "민수가 근무하던 작고 보이지 않는 부서의 코드명은 무엇인가?",
        "answer_doc_ids": [13, 14, 15],
    },

    # 4) [이유] 워커홀릭 개발자가 코딩하는 이유
    {
        "query": "워커홀릭 개발자(술주정뱅이)는 왜 계속 코딩을 하는가?",
        "answer_doc_ids": [44],
    },

    # 5) [원인-결과] 당직 근무자가 쉴 틈이 없는 이유
    {
        "query": "당직 근무자가 쉴 틈 없이 모니터를 껐다 켜야 하는 이유는 무엇인가?",
        "answer_doc_ids": [51, 52],
    },

    # 6) [개념 정의] 멘토가 말한 '원팀'의 의미
    {
        "query": "멘토(여우)가 말하는 '원팀(One Team)'이 된다는 것은 무슨 뜻인가?",
        "answer_doc_ids": [64, 65, 66],
    },

    # 7) [행동] 재무팀장이 자산을 가지고 하는 일
    {
        "query": "재무팀장은 5억 개의 자산(별)을 가지고 무엇을 하는가?",
        "answer_doc_ids": [46, 47, 48],
    },

    # 8) [원인/동기] 민수가 말한 '버그'의 존재 이유
    {
        "query": "민수는 왜 프로젝트가 버그를 만들어낸다고 설명했나?",
        "answer_doc_ids": [22, 23],
    },

    # 9) [결말 단서] 팀장이 깜빡 잊은 권한
    {
        "query": "팀장이 민수에게 준 AI 인턴에게 설정해 주는 것을 깜빡 잊은 권한은 무엇인가?",
        "answer_doc_ids": [87],
    },

    # 10) [번아웃 묘사] 번아웃일 때 보고 싶어진다고 한 화면
    {
        "query": "민수는 번아웃이 올 때 어떤 화면이 보고 싶어진다고 말했나?",
        "answer_doc_ids": [20],
    },

    # 11) [사건 원인] 메인 서버 다운의 직접 원인
    {
        "query": "지하 전산실에서 메인 서버가 다운된 직접 원인은 무엇이었나?",
        "answer_doc_ids": [4],
    },

    # 12) [구체 수량] 당시 남아 있던 커피 양
    {
        "query": "서버 복구 작업 중 팀장에게 남아 있던 커피(카페인)는 어느 정도였나?",
        "answer_doc_ids": [4],
    },

    # 13) [구체 수치] 로딩 바 100%를 본 횟수
    {
        "query": "민수는 로딩 바가 100% 되는 걸 몇 번이나 보았다고 했나?",
        "answer_doc_ids": [20],
    },

    # 14) [특정 시각] 본부장이 승인한 퇴근 시간
    {
        "query": "본부장은 민수의 퇴근을 오늘 저녁 몇 시에 승인하겠다고 했나?",
        "answer_doc_ids": [39],
    },

    # 15) [정의] 인플루언서 마케터가 말한 '리스펙'의 의미
    {
        "query": "인플루언서 마케터는 '리스펙'을 어떤 의미로 설명했나?",
        "answer_doc_ids": [43],
    },

    # 16) [정확한 숫자] 재무팀장이 말한 자산 액수
    {
        "query": "재무팀장이 말한 자산은 정확히 얼마라고 했나?",
        "answer_doc_ids": [45, 46],
    },

    # 17) [원인] 당직 근무자가 1초마다 새로고침해야 하는 이유
    {
        "query": "당직 근무자는 왜 1초에 한 번씩 새로고침을 해야 한다고 말했나?",
        "answer_doc_ids": [52],
    },

    # 18) [정책/이유] 기획 이사가 프로젝트를 기록하지 않는 이유
    {
        "query": "기획 이사는 왜 '핵심 프로젝트'를 기록하지 않는다고 했나?",
        "answer_doc_ids": [56, 57],
    },

    # 19) [주의 문장] 멘토가 슬랙 메시지에 대해 경고한 표현
    {
        "query": "멘토는 슬랙 메시지에 대해 어떤 경고를 했나?",
        "answer_doc_ids": [68],
    },

    # 20) [구체 사물] 팀장이 발견한 탕비실에 있던 설비
    {
        "query": "팀장이 발견한 탕비실에는 어떤 커피 머신과 어떤 정수기가 있었나?",
        "answer_doc_ids": [77],
    },
]

#### ② Recall@K 계산

In [ ]:
def recall_at_k(retriever, test_cases, k=5):
    correct = 0
    total = len(test_cases)

    for case in test_cases:
        retrieved = retrieve_ids(retriever, case["query"], k=k)
        if any(doc_id in retrieved for doc_id in case["answer_doc_ids"]):
            correct += 1

    return correct / total

#### ③ Hybrid / Dense / Sparse 비교 실험

In [ ]:
def debug_recall(retriever, test_cases, k=5):
    correct = 0
    total = len(test_cases)

    for case in test_cases:
        query = case["query"]
        gt_ids = case["answer_doc_ids"]
        
        # 모델 예측 (retrieved IDs)
        retrieved_docs = retriever.invoke(query)
        pred_ids = [d.metadata.get("id") for d in retrieved_docs[:k]]

        # 정답 여부 확인
        hit = any(doc_id in pred_ids for doc_id in gt_ids)
        if hit:
            correct += 1

        # ===== 디버깅 출력 =====
        print(f"\n[질문] {query}")
        print(f"- 예측 Top-{k} IDs: {pred_ids}")
        print(f"- 정답 IDs: {gt_ids}")
        print(f"- 정답 여부: {'O' if hit else 'X'}")
        print("-" * 50)

    print(f"\n총 Recall@{k}: {correct / total:.3f}")
debug_recall(retriever, test_cases, k=5)

In [ ]:
print("Dense Recall@5:", recall_at_k(dense_retriever, test_cases, k=5))
print("Sparse Recall@5:", recall_at_k(bm25_retriever, test_cases, k=5))
print("Hybrid Recall@5:", recall_at_k(hybrid_rrf, test_cases, k=5))


**관찰 포인트**

- Hybrid가 대부분 Recall에서 우수함  
- Sparse-only는 특정 키워드 문서에 강함  
- Dense-only는 표현이 달라진 경우에 강함  

### 6.3 Generation Evaluation (생성 품질 평가)

Generation은 다음 기준으로 평가한다.

✔ 1) Faithfulness (근거 기반 여부)  
- 답변이 문서에서 실제로 근거를 가지고 있는가?  
- 문서에 없는 내용을 hallucination으로 만들어내지 않았는가?

✔ 2) Relevance (질문과의 관련성)  
- 질문의 핵심을 정확히 답했는가?

✔ 3) Completeness  
- 문서에 있는 답을 빠짐없이 요약했는가?

✔ 4) Conciseness  
- 불필요하게 장황하지 않고 필요한 내용만 말했는가?

✔ 5) Citation Quality  
- 출처 인용이 정확한가?  
- 문서 번호/페이지 등 기반 정보가 연결되었는가?

#### Generation Evaluation 실습: LLM을 활용한 Generation 평가 (LLM-as-a-Judge)

LLM에게 “생성된 답변의 품질”을 직접 평가하게 할 수 있습니다.

In [ ]:
judge_prompt = """
다음은 RAG 모델이 생성한 답변과, 참조 문서(Context)입니다.
답변이 문서에 근거했는지 0~1 점수로 평가하세요.

- 1: 문서에서 근거를 명확히 가지고 있음
- 0: 문서에 없는 내용을 생성함 (hallucination)
- 0.5: 부분적으로만 근거가 있음

[질문]
{query}

[답변]
{answer}

[문맥]
{context}

점수만 출력하세요.
"""

from langchain_core.prompts import PromptTemplate
judge_template = PromptTemplate.from_template(judge_prompt)

def evaluate_answer(query, answer, context):
    judge = llm.invoke(judge_template.format(query=query, answer=answer, context=context))
    return float(judge.content.strip())

In [ ]:
query = "민수가 근무하던 작고 보이지 않는 부서의 코드명은 무엇인가?"

docs = hybrid_rrf.invoke(query)
context = format_context_with_ids(docs)

answer = llm.invoke(prompt.format(query=query, context=context)).content

score = evaluate_answer(query, answer, context)

print("=== 답변 ===")
print(answer)

print("\n=== Faithfulness Score ===")
print(score)

### 6.4 실무에서 사용하는 전체 평가 전략

기업이나 기관 등 대규모 환경에서는 다음과 같은 평가 절차를 종합적으로 수행합니다. 

1. **Retrieval 성능 평가**
   - Recall@K  
   - nDCG(순위 기반 품질 척도)  
   - Multi-query / Hybrid Retrieval 비교 분석  

2. **Generation 품질 평가**
   - LLM 기반 자동 평가(LLM-as-a-Judge)  
   - Human evaluation  
   - 생성된 답변의 근거 출처(citation) 검증  

3. **A/B 테스트**
   - 서로 다른 RAG 구성 중 어느 쪽이 더 우수한지 실험적으로 비교  

4. **Feedback Loop 구축**
   - 잘못된 답변 자동 수집  
   - 쿼리·문서 튜닝 및 재학습을 통한 지속적 개선

### 6.5 정리

1. **RAG 성능을 지속적으로 개선하는 유일한 방법은 체계적인 평가(evaluation)입니다.**  
2. Retrieval 평가에서는 문서 회수 능력을 나타내는 Recall@K가 가장 중요한 지표입니다.  
3. Generation 평가에서는 답변이 실제 문서에 근거하는지를 판단하는 Faithfulness가 핵심 기준입니다.  
4. LLM을 채점자(judge)로 활용하는 자동화된 평가 방식이 현재 가장 널리 사용됩니다.  
5. 이러한 평가 절차는 RAG 파이프라인을 비교·개선하고 품질을 안정적으로 높이는 데 필수 요소입니다.

## 7. RAG 운영 및 모니터링 (Monitoring & Observability)

### 7.1 왜 RAG에는 모니터링이 필요한가?

RAG 시스템은 단순한 LLM 호출이 아니라  
**검색(Retrieval)과 생성(Generation)이 결합된 파이프라인**입니다.

따라서 답변 품질이 저하되었을 때, 그 원인은 여러 단계 중 하나일 수 있습니다.

- 검색된 문서가 질문과 잘 맞지 않았는가?
- 검색 문서는 적절했지만 프롬프트 구성이 문제였는가?
- context 길이가 과도하게 길었는가?
- LLM 생성 과정에서 의미 왜곡이 발생했는가?

이러한 문제는 **최종 답변만 보고는 판단하기 어렵습니다.**  
RAG 시스템에서는 단순한 결과 평가를 넘어,  **과정 전체를 관찰할 수 있는 모니터링(Observability)** 이 필요합니다.

<img src="image/RAG.png" width="600">


### 7.2 RAG Observability 개념과 도구 선택

Observability란 시스템의 내부 상태를 외부에서 관찰 가능한 정보만으로 추론할 수 있는 능력을 의미합니다.

RAG 관점에서 Observability는 다음과 같은 질문에 답할 수 있어야 합니다.

- 어떤 질문이 입력되었는가?
- Retriever는 어떤 문서들을 반환했는가?
- 실제 LLM에 전달된 context는 무엇이었는가?
- 응답 생성까지 어느 단계에서 시간이 소요되었는가?

이를 위해 RAG 시스템에서는 다음과 같은 도구들이 활용됩니다.

- LangSmith
- Langfuse

이러한 도구들은 모두 **RAG 요청 단위의 Trace를 수집하고 분석**하는 것을 목표로 합니다.

### 7.3 LangSmith vs Langfuse 비교

LangSmith와 Langfuse는 목적은 유사하지만, 성격과 활용 환경에는 차이가 있습니다.

### 공통점
- RAG 요청 단위 추적
- Prompt / Context / Output 기록
- 성능 분석을 위한 로그 제공

### 차이점 개요

| 구분 | LangSmith | Langfuse |
|-----|----------|----------|
| 개발 주체 | LangChain | Langfuse |
| 프레임워크 의존성 | LangChain 중심 | 프레임워크 독립 |
| 배포 방식 | 관리형 중심 | 오픈소스 / 자체 호스팅 가능 |
| 기업 환경 적합성 | 빠른 실험용 | 내부 시스템 연동에 유리 |

Langfuse는 **오픈소스 기반**이며, 자체 서버에 설치하여 운영할 수 있기 때문에  
기업 환경에서 RAG 시스템을 운영할 때 자주 선택됩니다.

본 강의에서는 **실무 환경에서 활용하기 적합한 RAG 모니터링 도구의 예시**로  
Langfuse를 사용합니다.

## 7.4 Langfuse 시작하기

### 1) Langfuse 회원가입 및 조직(Organization) 생성
- https://langfuse.com 접속
- 회원가입 진행
- 로그인 후 Organization 생성  
  (Langfuse는 모든 프로젝트가 조직 단위로 관리됨)
  
  <img src="image/langfuse_organization.png" width=650>  
  
  <br>
  
  <img src="image/langfuse_organization2.png" width=650>

### 2) 프로젝트(Project) 생성  

- 생성한 Organization 내부에서 새 프로젝트 생성  
- 프로젝트 이름 및 기본 설정 입력  

  <img src="image/langfuse_project.png" width=650>

### 3) API Key 발급
- Project Settings 메뉴 이동  

  <img src="image/langfuse_api_key.png" width=650>

- Public Key / Secret Key 확인  

  <img src="image/langfuse_api_key2.png" width=650>

### 4) .env 파일에 환경 변수 설정

```bash
LANGFUSE_PUBLIC_KEY=...
LANGFUSE_SECRET_KEY=...
LANGFUSE_HOST=https://cloud.langfuse.com
```

이후 Python 코드에서 load_dotenv 함수를 통해 환경 변수를 로드하여 사용합니다.

```python
from dotenv import load_dotenv
load_dotenv()
```

본 장에서는 위에서 이미 구축한 벡터 DB와 `dense_retriever`를 이용하여,  
 **RAG 체인 실행 결과를 Langfuse에 기록하는 방법**에 집중합니다.


### 7.5 실습: Langfuse CallbackHandler로 RAG 요청 추적하기

Langfuse의 `CallbackHandler`는  
**체인 실행 시점에 전달하여 전체 실행 흐름을 추적**하는 방식으로 사용합니다.

이 방식을 사용할 경우,

- Retriever 검색 단계
- Context 구성
- Prompt 적용
- LLM 호출
- 최종 Output 생성

까지 **RAG 파이프라인 전체가 하나의 Trace로 Langfuse에 기록**됩니다.

#### 1) Langfuse CallbackHandler 생성

In [1]:
from langfuse.langchain import CallbackHandler
langfuse_handler = CallbackHandler()

#### 2) RAG 체인 실행 (CallbackHandler 전달)

이미 구성된 LCEL 기반 RAG 체인을 실행할 때,  
`config` 인자를 통해 CallbackHandler를 전달합니다.

In [2]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS

# 문서 리스트를 하나의 문자열 context로 변환
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# RAG 프롬프트
rag_prompt = ChatPromptTemplate.from_template(
    """
    다음 문서를 참고하여 질문에 답변하세요.
    문서에 없는 내용은 "없는 정보"라고 답변하세요.

    [문서]
    {context}

    [질문]
    {question}
    """
)

# LLM 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",
    
    temperature=0
)

# FAISS 벡터 스토어 로드
DB_PATH = "./faiss_index_minsu"
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = FAISS.load_local(
    folder_path=DB_PATH,
    embeddings=embeddings,
    allow_dangerous_deserialization=True  # 필수 설정
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# LCEL 기반 RAG 체인 구성
rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever | format_docs,
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

# 질의 실행 (Langfuse CallbackHandler 연결)
question = "신입사원 민수가 세 번째 부서에서 만난 개발자는 왜 쉬지 않고 일했나요?"

answer = rag_chain.invoke(
    question,
    config={
        "callbacks": [langfuse_handler]
    }
)

print(answer)


신입사원 민수가 세 번째 부서에서 만난 개발자는 "부끄럽다는 걸 잊기 위해서" 코딩을 하고 있다고 말했습니다.


이 방식으로 실행하면, 
Retriever → Prompt → LLM → Output 전체 흐름이 Langfuse에 자동으로 저장됩니다.

<img src="image/langfuse_tracing.png" width="800">

### 7.6 Langfuse 주요 기능 살펴보기

Langfuse는 단순히 하나의 RAG 요청 결과만 확인하는 도구가 아니라, RAG 시스템의 실행 흐름을 기록하고, 결과 품질을 평가하며,  
프롬프트와 데이터셋을 기반으로 실험과 개선을 반복할 수 있도록 다양한 관측(Observability) 기능을 제공합니다.

아래 표는 Langfuse에서 제공하는 주요 기능과 각 기능의 역할을 정리한 것입니다.

| 영역                    | 기능               | 설명                                          | 활용 목적            |
| --------------------- | ---------------- | ------------------------------------------- | ---------------- |
| **Tracing**           | Trace            | 하나의 사용자 요청(RAG 요청) 단위 실행 흐름                 | 요청 단위 디버깅, 성능 비교 |
|                       | Observation      | Trace 내부의 개별 실행 단계 (LLM 호출, Retriever 실행 등) | 단계별 병목 분석        |
| **Sessions / Users**  | Sessions         | 여러 요청을 하나의 세션으로 묶어 관리                       | 사용자 흐름 분석        |
|                       | Users            | 사용자 단위 실행 기록 관리                             | 사용자별 사용 패턴 분석    |
| **Prompt Management** | Prompts          | 프롬프트 템플릿을 자산으로 관리                           | 프롬프트 버전 관리, 실험   |
| **Evaluation**        | Scores           | 출력 결과에 점수 부여                                | 품질 정량 평가         |
|                       | LLM-as-a-Judge   | 다른 LLM을 평가자로 활용                             | 자동화된 대규모 평가      |
|                       | Human Annotation | 사람이 직접 결과를 평가                               | 정성 평가, 기준 검증     |
|                       | Datasets         | 질문·정답 기준 데이터셋 관리                            | 반복 실험, 성능 비교     |
| **Dashboards**        | Dashboards       | 주요 지표를 요약해 시각화                              | 운영 상태 모니터링       |


<img src="image/langfuse_main.png" width="800">


### 7.7 Langfuse를 활용한 품질 분석 사례

Langfuse에 기록된 로그를 통해 다음과 같은 분석이 가능합니다.

- 답변이 부정확한 요청에서
  - Retriever 결과가 적절했는지
  - Context가 과도하게 길지는 않았는지
- 동일한 질문에 대해
  - 서로 다른 프롬프트 버전이 어떤 영향을 주는지
- 응답 품질 저하가
  - 검색 단계 문제인지
  - 생성 단계 문제인지

이는 **정량 평가 점수 없이도**  
문제의 원인을 구조적으로 추적할 수 있게 해줍니다.

### 7.8 평가(Evaluation)와 모니터링의 역할 차이

Evaluation과 Monitoring은 서로 다른 목적을 가집니다.

- Evaluation
  - 결과가 좋은지 판단
  - 정답률, 점수 중심
- Monitoring
  - 왜 이런 결과가 나왔는지 분석
  - 과정과 맥락 중심

실무 환경에서는 두 접근 방식이 함께 사용됩니다.

- Evaluation은 기준을 만들고
- Monitoring은 개선 방향을 찾는 역할을 합니다.

### 7.9 정리

- RAG 시스템은 한 번 구축하고 끝나는 시스템이 아닙니다.
- 운영 과정에서 지속적인 관찰과 분석이 필요합니다.
- Langfuse와 같은 Observability 도구를 활용하면
  - RAG 내부 동작을 이해할 수 있고
  - 성능 문제를 체계적으로 개선할 수 있습니다.